<font size="8">Homework 2: AirBnB Document Database</font>

Group number: `3`

Group members:
1. `Julia Karpienia : 20240514`
2. `Elizaveta Nosova : 20242132`
3. `Zofia Wójcik : 20240654`

The Homework 2 is comprised of two parts:
1. Data modelling (15 points).
2. Queries to database to answer the questions (25 points).

<font size="6">1. Data Modelling</font>

<font size="4">Congratulations! You’ve been hired as part of the new Data Engineering and Management team in the AirBNB Business Intelligence department. The company is restructuring due to unsatisfactory performance from the previous teams.

Before leaving, the head of the Data Modelling department highlighted several issues:

**Data Storage**: A lot of data about AirBNB listings is stored in a single document. While this approach has some advantages, it has also caused performance issues. Queries are slow, and the team didn’t apply patterns, which could improve performance by optimizing the data model. Indexes were also not used.

**Reviews Growth**: The number of reviews for AirBNB is growing rapidly. Currently, we overwrite reviews regularly, but the Business Intelligence department will benefit from storing all reviews and analyzing them over time.

**Data Errors**: There are errors in the data collection, such as duplicate data entries and incorrect timestamps for transactions. The new team will need to decide how to fix these issues.

**Your Role**: In your new role, you’ll need to consider how each database query is used, how often it is needed, and its impact on reads and writes. You should update the database schema to optimize for business use cases. Use tools like embedding, linking, indexes, and patterns to improve the data model. You may need to create new fields, documents, or collections. Be sure to document the pattern you’re applying and the reasons behind your decisions, especially when dealing with duplication and risks of outdated data.

**Key tasks include**:

1. Streamlining the data collection process.
2. Cleaning up the data and optimizing what will be returned for each use case.
3. Applying the correct patterns to speed up common queries.
4. Ensuring departments get accurate and relevant information from the database.
5. Sharing the updated data model schema with other departments.

**Good Practices**: [Check Chapter 6, Mastering MongoDB]

1. All newly created fields should have capitalized names.
2. New queries should work with the most up-to-date database version. If you make multiple changes, all queries should still work after the final updates.
3. For some queries, you may need to change the database schema.
4. When you are applying specific patterns, like polymorphic, subset, or bucket, name them accordingly. 
5. Document each major transformation using this format:
*“We applied {transformation name} because {reasoning behind it}. We expect {change/result} based on {observable measure, such as query speed, number of documents returned, index use, etc.}.”*

</font>


**Data Cleanup and Schema Adjustments:** [9 points in total]

1) Before working on the queries below, review the data and adjust the schema based on the typical use case described.

**Typical Use Case**: The most common use of the database is to show property listing information to customers. A query retrieves a listing document from the database. Currently, retrieving a listing takes too long. Decide what information should be included in a typical query and optimize the structure accordingly. For example, customers usually only need a sample of reviews, not all reviews (even though all reviews are stored). They also don’t need past transaction data. Update the document schema to fit this use case. This might involve creating new collections or documents.

**Data Cleanup**: Review the data for any errors (such as transactions that don’t belong to the listing) or unnecessary duplication, and clean it up where needed.

<font size="6">2. Uses for the database</font>

<font size="4">Different AirBNB departments require different analytics from our common database.
Below are the specific questions from various departments: 

**Standard Difficulty Questions:** [2 points per question]

2)	Once a month, we reward hosts with recognition. Select three superhosts with at least two listings that can accommodate more than four people.

3)	The company considers investing into property to rent. Which bed type is most common in listings with a waterfront and a dishwasher in New York?

4)	We're considering hiring someone to write reviews professionally. Who wrote the longest review in New York?

5)	To assess the security of different areas, what is the biggest and smallest (price-security deposit) difference per number of visitors at a property?

6)  Identify areas by whether they are typically used for short breaks, like weekend mini breaks, or whether they are more suitable for long trips. This information support targeted advertising of different customer types. It is not expected to change much over time so we won’t look to update it, we just require current view. What is the average duration of stay (in nights) per type of property per city (you can use the maximum_nights to measure length of stays)? For each property type return the city with the highest and lowest average value.

**Advanced Difficulty Questions (Consider database optimization for these queries):** [3 points per question]

7)	We are creating a new webpage for hosts when setting up their account. It will list suggested typical amenities. This data will need to be available every time a host registers a property but is not expected to change very much. The starting point for the list will be all unique amenities currently listed in properties (across all documents). Optimise the database for this use case and show how the data should be queried.

8)	We plan to track our reviewers better. We want to create a webpage that shows the top 20 reviewers and the count of the number of reviews of each of these reviewers. This webpage should be kept up to date. It should also have a link to return the number of reviews for a given reviewer ID or Name (show how to query for number of reviews by ID or query quickly).

9)	For each property we store review scores across different metrics (accuracy, check-in, cleanliness etc). We consider adding more metrics, although there is no clarity on what these will be. We want to be able to easily query the average score across all of these metrics, including any new metrics that might be added without changing the query. Adjust the data model so this can be done and show the query for an example property.

10)	We aim to have better access to information about transaction, we wish to develop a search engine that can calculate the average value of transactions in a given period of time quickly for a given property.

11)	We wish to have a summary webpage that displays information about our top destinations. This webpage should display for each of the top 10 cities some basic information about our operations in the area (number of properties by type for example, average price by type) but you can choose the metrics. For each of the top 10 cities it should also provide some basic information about the top 3 properties in each city (price, number of review, whatever you think useful) to show an example of the properties available in the area. We would like to keep this webpage up to date as information changes.

**Database updates:** [2 points per question]

After optimizing the database, show how to complete the following updates. You can create fictional data. Ensure that previous data does not become stale:

12) Add a new property with a new host in one of the top 10 cities. The host selects the top 10 most common amenities to list.

13) Add a new review from one of our top 20 reviewers for this new property.

14) Add a new review metric called 'x_factor' with a score of 10. Show that the average score across all metrics is correctly calculated for this listing, using the previously developed query.


In [1]:
# Python Connector

# ! pip install pymongo
# or #!conda install -y pymongo

from datetime import datetime
from pprint import pprint
import time
import re
from bson.objectid import ObjectId
from bson.decimal128 import Decimal128
from collections import defaultdict

from pymongo import UpdateMany
from pymongo import MongoClient
from pymongo import UpdateOne

from collections import Counter
import random

user="AzureDiamond"
password="hunter2"
host="localhost"
port="27017"
protocol="mongodb"

client = MongoClient(f"{protocol}://{user}:{password}@{host}:{port}")

# Database check
db = client.sample_airbnb
print(f"Database info: {db}\n")
db.name 

Database info: Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'sample_airbnb')



'sample_airbnb'

<font size="6">1. Data Modelling</font>

## Loading database

In [ ]:
# Collections are inside our Database 'sample_analytics' 

collection_list = db.list_collection_names()

print(f"The database contains {len(collection_list)} collections")
print(f"All collections: {collection_list[0:]}")
print(f"Collection {collection_list[0]} contains {db[collection_list[0]].count_documents({})} documents")

The database contains 1 collections
All collections: ['listingsAndReviews_new']
Collection listingsAndReviews_new contains 5555 documents


In [3]:
# define the original collection
raw_col = db.listingsAndReviews_new

---

## Data Exploration

In [4]:
# Show all top-level fields in one listing
sample_doc = db.listingsAndReviews_new.find_one()
pprint(sample_doc)

{'_id': '10006546',
 'access': 'We are always available to help guests. The house is fully '
           'available to guests. We are always ready to assist guests. when '
           'possible we pick the guests at the airport.  This service transfer '
           'have a cost per person. We will also have service "meal at home" '
           'with a diverse menu and the taste of each. Enjoy the moment!',
 'accommodates': 8,
 'address': {'country': 'Portugal',
             'country_code': 'PT',
             'government_area': 'Cedofeita, Ildefonso, Sé, Miragaia, Nicolau, '
                                'Vitória',
             'location': {'coordinates': [-8.61308, 41.1413],
                          'is_location_exact': False,
                          'type': 'Point'},
             'market': 'Porto',
             'street': 'Porto, Porto, Portugal',
             'suburb': ''},
 'amenities': ['TV',
               'Cable TV',
               'Wifi',
               'Kitchen',
              

In [5]:
# Inspect unique values in each field 

all_fields = raw_col.find_one()

# Loop over fields and print distinct values
for field in sorted(set(all_fields)):
    try:
        values = raw_col.distinct(field)
        print(f"\nField: {field} | Unique values: {len(values)}")
        pprint(values[:20])
        if len(values) > 20:
            print("...and more.")
    except Exception as e:
        print(f"Error in field {field}: {e}")


Field: _id | Unique values: 5555
['10006546',
 '10009999',
 '1001265',
 '10021707',
 '10030955',
 '1003530',
 '10038496',
 '10047964',
 '10051164',
 '10057447',
 '10057826',
 '10059244',
 '10059872',
 '10066928',
 '10069642',
 '10082307',
 '10082422',
 '10083468',
 '10084023',
 '10091713']
...and more.

Field: access | Unique values: 2989
['',
 ' Everything is located in a beautiful and spacious flat, shared with our 4 '
 'other nice roomates. Enjoy our large airy space in the basement, including '
 'the principal room with a large double bed, a office with a sofa bed, as '
 'well as your own private bathroom.  The roomates can pass in the principal '
 'room to acces on the washing room. You could access to the kitchen, the '
 'living room, the garden, and all the common space!',
 ' ROOMS WITH AIR CONDITIONING, 3 BATHROOMS, LARGE LIVING ROOM, GREAT BALCONY, '
 '2 LCD TVs, FRIDGE, STOVE, DISHWASHER, MICROWAVE, 8 POOL, GYM, BAR, BAKERY , '
 'TENNIS COURTS, BASKETBALL COURT.',
 '(SENSITI

* accommodates

In [6]:
counter = Counter()

for doc in raw_col.find({"accommodates": {"$exists": True}}, {"accommodates": 1}):
    counter[str(doc["accommodates"])] += 1

for val, count in counter.most_common():
    print(f"{val}: {count} listings")

2: 2052 listings
4: 1154 listings
3: 567 listings
1: 561 listings
6: 505 listings
5: 264 listings
8: 191 listings
7: 83 listings
10: 67 listings
12: 31 listings
9: 30 listings
16: 23 listings
14: 11 listings
11: 7 listings
13: 6 listings
15: 3 listings


No weird, unusual values.

* bathrooms

In [7]:
counter = Counter()

for doc in raw_col.find({"bathrooms": {"$exists": True}}, {"bathrooms": 1}):
    counter[str(doc["bathrooms"])] += 1

for val, count in counter.most_common():
    print(f"{val}: {count} listings")

1.0: 4202 listings
2.0: 774 listings
1.5: 231 listings
3.0: 132 listings
2.5: 79 listings
4.0: 33 listings
3.5: 26 listings
0.5: 15 listings
0.0: 14 listings
5.0: 11 listings
4.5: 8 listings
7.0: 6 listings
6.0: 5 listings
8.0: 5 listings
5.5: 2 listings
16.0: 1 listings
9.0: 1 listings


There are listing with 0 bathrooms, which can seem weird but it can also states that there is a shared bathroom or no bathroom. Half numbers are possible because it usually states that there is a bathroom and small toilet.

* bedrooms

In [8]:
counter = Counter()

for doc in raw_col.find({"bedrooms": {"$exists": True}}, {"bedrooms": 1}):
    counter[str(doc["bedrooms"])] += 1

for val, count in counter.most_common():
    print(f"{val}: {count} listings")

1: 3308 listings
2: 1090 listings
0: 496 listings
3: 427 listings
4: 161 listings
5: 36 listings
6: 16 listings
7: 7 listings
8: 3 listings
9: 2 listings
10: 2 listings
20: 1 listings
15: 1 listings


There are some listings with 0 bedrooms, which can state that it's a studio apartment or the apartment is not furnished or guests will sleep on something else than bed e.g couch

* beds

In [9]:
counter = Counter()

for doc in raw_col.find({"beds": {"$exists": True}}, {"beds": 1}):
    counter[str(doc["beds"])] += 1

for val, count in counter.most_common():
    print(f"{val}: {count} listings")

1: 2700 listings
2: 1381 listings
3: 657 listings
4: 385 listings
5: 166 listings
6: 109 listings
7: 43 listings
0: 29 listings
8: 28 listings
10: 16 listings
9: 11 listings
12: 5 listings
11: 3 listings
15: 2 listings
14: 2 listings
16: 2 listings
25: 1 listings
13: 1 listings
18: 1 listings


No weird values

* cleaning_fee

In [10]:
counter = Counter()

for doc in raw_col.find({"cleaning_fee": {"$exists": True}}, {"cleaning_fee": 1}):
    fee = doc.get("cleaning_fee")
    if fee is not None:
        counter[str(fee)] += 1

for val, count in sorted(counter.items(), key=lambda x: float(x[0].replace("$", "").replace(",", ""))):
    print(f"{val}: {count} listings")


0.00: 400 listings
4.00: 1 listings
5.00: 35 listings
6.00: 8 listings
7.00: 9 listings
8.00: 19 listings
9.00: 1 listings
10.00: 144 listings
11.00: 1 listings
12.00: 14 listings
14.00: 5 listings
15.00: 136 listings
16.00: 6 listings
17.00: 2 listings
18.00: 6 listings
19.00: 7 listings
20.00: 202 listings
22.00: 6 listings
23.00: 5 listings
24.00: 3 listings
25.00: 133 listings
26.00: 11 listings
27.00: 4 listings
28.00: 5 listings
29.00: 11 listings
30.00: 153 listings
31.00: 2 listings
32.00: 2 listings
33.00: 1 listings
34.00: 1 listings
35.00: 65 listings
36.00: 2 listings
37.00: 4 listings
38.00: 1 listings
39.00: 11 listings
40.00: 110 listings
41.00: 1 listings
42.00: 7 listings
45.00: 39 listings
46.00: 2 listings
47.00: 3 listings
48.00: 5 listings
49.00: 11 listings
50.00: 297 listings
51.00: 1 listings
52.00: 1 listings
53.00: 5 listings
54.00: 1 listings
55.00: 34 listings
56.00: 1 listings
57.00: 2 listings
58.00: 3 listings
59.00: 6 listings
60.00: 133 listings
62.00: 

In [11]:
counter = Counter()

for doc in raw_col.find({"cleaning_fee": {"$exists": True}}, {"cleaning_fee": 1}):
    counter[str(doc["cleaning_fee"])] += 1

for val, count in counter.most_common():
    print(f"{val}: {count} listings")

0.00: 400 listings
50.00: 297 listings
100.00: 215 listings
150.00: 209 listings
20.00: 202 listings
30.00: 153 listings
10.00: 144 listings
15.00: 136 listings
60.00: 133 listings
25.00: 133 listings
80.00: 111 listings
40.00: 110 listings
200.00: 105 listings
75.00: 90 listings
120.00: 73 listings
70.00: 70 listings
35.00: 65 listings
250.00: 57 listings
300.00: 53 listings
90.00: 45 listings
125.00: 44 listings
45.00: 39 listings
85.00: 37 listings
65.00: 37 listings
5.00: 35 listings
55.00: 34 listings
180.00: 30 listings
130.00: 28 listings
99.00: 27 listings
160.00: 26 listings
140.00: 24 listings
350.00: 24 listings
110.00: 24 listings
175.00: 23 listings
95.00: 21 listings
8.00: 19 listings
400.00: 18 listings
105.00: 18 listings
69.00: 16 listings
135.00: 14 listings
12.00: 14 listings
89.00: 14 listings
149.00: 12 listings
26.00: 11 listings
500.00: 11 listings
29.00: 11 listings
49.00: 11 listings
39.00: 11 listings
187.00: 10 listings
79.00: 10 listings
7.00: 9 listings
179

There are some extremely high cleaning fees such as 2000 (only one listing). No negative values.

* extra_people

In [12]:
counter = Counter()

for doc in raw_col.find({"extra_people": {"$exists": True}}, {"extra_people": 1}):
    counter[str(doc["extra_people"])] += 1  

for val, count in sorted(counter.items(), key=lambda x: float(x[0].replace("$", "").replace(",", ""))):
    print(f"{val}: {count} listings")

0.00: 3135 listings
4.00: 10 listings
5.00: 58 listings
6.00: 8 listings
7.00: 36 listings
8.00: 24 listings
9.00: 13 listings
10.00: 295 listings
11.00: 6 listings
12.00: 30 listings
13.00: 6 listings
14.00: 4 listings
15.00: 252 listings
16.00: 2 listings
17.00: 10 listings
18.00: 11 listings
19.00: 10 listings
20.00: 281 listings
21.00: 4 listings
22.00: 14 listings
23.00: 1 listings
24.00: 4 listings
25.00: 223 listings
26.00: 11 listings
27.00: 6 listings
28.00: 2 listings
29.00: 9 listings
30.00: 143 listings
31.00: 4 listings
32.00: 4 listings
34.00: 3 listings
35.00: 55 listings
36.00: 7 listings
37.00: 3 listings
39.00: 4 listings
40.00: 65 listings
42.00: 11 listings
44.00: 3 listings
45.00: 19 listings
46.00: 1 listings
48.00: 3 listings
49.00: 3 listings
50.00: 219 listings
52.00: 2 listings
53.00: 13 listings
54.00: 1 listings
55.00: 10 listings
58.00: 4 listings
59.00: 2 listings
60.00: 36 listings
63.00: 3 listings
65.00: 4 listings
66.00: 1 listings
68.00: 1 listings
70

No negative values. 

* guests_included

In [13]:
counter = Counter()

for doc in raw_col.find({"guests_included": {"$exists": True}}, {"guests_included": 1}):
    counter[str(doc["guests_included"])] += 1  

for val, count in sorted(counter.items(), key=lambda x: float(x[0].replace("$", "").replace(",", ""))):
    print(f"{val}: {count} listings")

1: 3595 listings
2: 1165 listings
3: 155 listings
4: 369 listings
5: 59 listings
6: 127 listings
7: 14 listings
8: 33 listings
9: 6 listings
10: 18 listings
12: 7 listings
13: 2 listings
15: 2 listings
16: 3 listings


No strange values.

* host_listings_count

In [14]:
counter = Counter()

for doc in raw_col.find({"host_listings_count": {"$exists": True}}, {"host_listings_count": 1}):
    counter[str(doc["host_listings_count"])] += 1  

for val, count in sorted(counter.items(), key=lambda x: float(x[0].replace("$", "").replace(",", ""))):
    print(f"{val}: {count} listings")

0: 5 listings
1: 2505 listings
2: 824 listings
3: 384 listings
4: 240 listings
5: 166 listings
6: 140 listings
7: 116 listings
8: 95 listings
9: 73 listings
10: 49 listings
11: 48 listings
12: 65 listings
13: 43 listings
14: 30 listings
15: 34 listings
16: 28 listings
17: 36 listings
18: 28 listings
19: 25 listings
20: 18 listings
21: 16 listings
22: 21 listings
23: 18 listings
24: 15 listings
25: 7 listings
26: 12 listings
27: 5 listings
28: 17 listings
29: 8 listings
30: 10 listings
31: 16 listings
32: 4 listings
33: 14 listings
34: 8 listings
35: 8 listings
36: 9 listings
37: 12 listings
38: 11 listings
39: 6 listings
40: 8 listings
41: 5 listings
42: 15 listings
43: 6 listings
44: 7 listings
45: 5 listings
46: 10 listings
47: 4 listings
48: 2 listings
49: 5 listings
50: 6 listings
51: 9 listings
52: 3 listings
53: 7 listings
54: 5 listings
55: 1 listings
56: 4 listings
57: 3 listings
58: 5 listings
59: 12 listings
60: 7 listings
61: 7 listings
62: 4 listings
63: 7 listings
64: 1 li

There are 5 hosts that does not have any listing on the platform

* host_response_rate

In [15]:
counter = Counter()

for doc in raw_col.find({"host_response_rate": {"$exists": True}}, {"host_response_rate": 1}):
    fee = doc.get("host_response_rate")
    if fee is not None:
        counter[str(fee)] += 1

for val, count in sorted(counter.items(), key=lambda x: float(x[0].replace("$", "").replace(",", ""))):
    print(f"{val}: {count} listings")

0: 97 listings
10: 3 listings
17: 1 listings
19: 2 listings
20: 3 listings
25: 3 listings
29: 2 listings
30: 1 listings
33: 12 listings
38: 2 listings
40: 8 listings
42: 1 listings
43: 2 listings
44: 5 listings
46: 1 listings
47: 1 listings
50: 61 listings
52: 1 listings
54: 1 listings
55: 1 listings
56: 7 listings
57: 3 listings
58: 1 listings
59: 1 listings
60: 30 listings
62: 1 listings
63: 2 listings
64: 9 listings
65: 1 listings
66: 1 listings
67: 45 listings
68: 1 listings
70: 30 listings
71: 8 listings
72: 1 listings
73: 3 listings
74: 2 listings
75: 26 listings
76: 9 listings
78: 7 listings
79: 5 listings
80: 77 listings
81: 5 listings
82: 8 listings
83: 25 listings
84: 1 listings
85: 11 listings
86: 17 listings
87: 9 listings
88: 30 listings
89: 11 listings
90: 133 listings
91: 18 listings
92: 52 listings
93: 33 listings
94: 52 listings
95: 41 listings
96: 35 listings
97: 43 listings
98: 109 listings
99: 104 listings
100: 2952 listings


* host_total_listings_count

In [16]:
counter = Counter()

for doc in raw_col.find({"host_total_listings_count": {"$exists": True}}, {"host_total_listings_count": 1}):
    counter[str(doc["host_total_listings_count"])] += 1  

for val, count in sorted(counter.items(), key=lambda x: float(x[0].replace("$", "").replace(",", ""))):
    print(f"{val}: {count} listings")

0: 5 listings
1: 2505 listings
2: 824 listings
3: 384 listings
4: 240 listings
5: 166 listings
6: 140 listings
7: 116 listings
8: 95 listings
9: 73 listings
10: 49 listings
11: 48 listings
12: 65 listings
13: 43 listings
14: 30 listings
15: 34 listings
16: 28 listings
17: 36 listings
18: 28 listings
19: 25 listings
20: 18 listings
21: 16 listings
22: 21 listings
23: 18 listings
24: 15 listings
25: 7 listings
26: 12 listings
27: 5 listings
28: 17 listings
29: 8 listings
30: 10 listings
31: 16 listings
32: 4 listings
33: 14 listings
34: 8 listings
35: 8 listings
36: 9 listings
37: 12 listings
38: 11 listings
39: 6 listings
40: 8 listings
41: 5 listings
42: 15 listings
43: 6 listings
44: 7 listings
45: 5 listings
46: 10 listings
47: 4 listings
48: 2 listings
49: 5 listings
50: 6 listings
51: 9 listings
52: 3 listings
53: 7 listings
54: 5 listings
55: 1 listings
56: 4 listings
57: 3 listings
58: 5 listings
59: 12 listings
60: 7 listings
61: 7 listings
62: 4 listings
63: 7 listings
64: 1 li

There are 11 listings where total listing count is high (1198) 

* number_of_reviews

In [17]:
counter = Counter()

for doc in raw_col.find({"number_of_reviews": {"$exists": True}}, {"number_of_reviews": 1}):
    counter[str(doc["number_of_reviews"])] += 1  

for val, count in sorted(counter.items(), key=lambda x: float(x[0].replace("$", "").replace(",", ""))):
    print(f"{val}: {count} listings")

0: 1388 listings
1: 511 listings
2: 329 listings
3: 247 listings
4: 187 listings
5: 128 listings
6: 139 listings
7: 107 listings
8: 88 listings
9: 61 listings
10: 69 listings
11: 70 listings
12: 71 listings
13: 58 listings
14: 53 listings
15: 56 listings
16: 59 listings
17: 40 listings
18: 49 listings
19: 46 listings
20: 42 listings
21: 34 listings
22: 37 listings
23: 37 listings
24: 38 listings
25: 35 listings
26: 21 listings
27: 35 listings
28: 28 listings
29: 28 listings
30: 20 listings
31: 33 listings
32: 34 listings
33: 24 listings
34: 21 listings
35: 35 listings
36: 23 listings
37: 30 listings
38: 21 listings
39: 23 listings
40: 29 listings
41: 18 listings
42: 17 listings
43: 18 listings
44: 19 listings
45: 24 listings
46: 21 listings
47: 22 listings
48: 20 listings
49: 15 listings
50: 11 listings
51: 15 listings
52: 10 listings
53: 15 listings
54: 14 listings
55: 16 listings
56: 17 listings
57: 15 listings
58: 12 listings
59: 12 listings
60: 12 listings
61: 17 listings
62: 22 li

There are not number of reviews below 0. Majority number of reviews is 0. 

* price

In [18]:
counter = Counter()

for doc in raw_col.find({"price": {"$exists": True}}, {"price": 1}):
    price = doc["price"]
    if isinstance(price, Decimal128):
        price = float(price.to_decimal())  # safe conversion from Decimal128 to float
    counter[price] += 1

# Sort and display prices from lowest to highest
for val in sorted(counter):
    print(f"{val}: {counter[val]} listings")

9.0: 1 listings
10.0: 3 listings
12.0: 2 listings
13.0: 4 listings
14.0: 5 listings
15.0: 19 listings
16.0: 7 listings
17.0: 10 listings
18.0: 7 listings
19.0: 5 listings
20.0: 24 listings
21.0: 7 listings
22.0: 9 listings
23.0: 12 listings
24.0: 9 listings
25.0: 48 listings
26.0: 8 listings
27.0: 9 listings
28.0: 10 listings
29.0: 11 listings
30.0: 63 listings
31.0: 13 listings
32.0: 18 listings
33.0: 9 listings
34.0: 11 listings
35.0: 72 listings
36.0: 9 listings
37.0: 10 listings
38.0: 20 listings
39.0: 27 listings
40.0: 81 listings
41.0: 10 listings
42.0: 20 listings
43.0: 12 listings
44.0: 9 listings
45.0: 65 listings
46.0: 27 listings
47.0: 21 listings
48.0: 18 listings
49.0: 17 listings
50.0: 126 listings
51.0: 9 listings
52.0: 7 listings
53.0: 24 listings
54.0: 7 listings
55.0: 71 listings
56.0: 9 listings
57.0: 4 listings
58.0: 35 listings
59.0: 21 listings
60.0: 137 listings
61.0: 4 listings
62.0: 5 listings
63.0: 24 listings
64.0: 22 listings
65.0: 54 listings
66.0: 7 listin

There is no price which is below 0. But there are some listings with really high price (> 10 000 - 6 listings)

* property_type

In [19]:
counter = Counter()

for doc in raw_col.find({"property_type": {"$exists": True}}, {"property_type": 1}):
    counter[str(doc["property_type"])] += 1

for val, count in counter.most_common():
    print(f"{val}: {count} listings")

Apartment: 3626 listings
House: 606 listings
Condominium: 399 listings
Serviced apartment: 185 listings
Loft: 142 listings
Townhouse: 108 listings
Guest suite: 81 listings
Bed and breakfast: 69 listings
Boutique hotel: 53 listings
Guesthouse: 50 listings
Hostel: 34 listings
Villa: 32 listings
Hotel: 26 listings
Aparthotel: 23 listings
Cottage: 20 listings
Other: 18 listings
Cabin: 15 listings
Bungalow: 14 listings
Resort: 11 listings
Farm stay: 9 listings
Casa particular (Cuba): 9 listings
Tiny house: 7 listings
Chalet: 2 listings
Boat: 2 listings
Nature lodge: 2 listings
Camper/RV: 2 listings
Treehouse: 1 listings
Earth house: 1 listings
Barn: 1 listings
Hut: 1 listings
Heritage hotel (India): 1 listings
Pension (South Korea): 1 listings
Campsite: 1 listings
Houseboat: 1 listings
Castle: 1 listings
Train: 1 listings


* maximum_nights

In [20]:
counter = Counter()

for doc in raw_col.find({"maximum_nights": {"$exists": True}}, {"maximum_nights": 1}):
    counter[str(doc["maximum_nights"])] += 1  

for val, count in sorted(counter.items(), key=lambda x: float(x[0].replace("$", "").replace(",", ""))):
    print(f"{val}: {count} listings")

1: 5 listings
2: 13 listings
3: 24 listings
4: 24 listings
5: 47 listings
6: 32 listings
7: 121 listings
8: 20 listings
9: 16 listings
10: 78 listings
11: 8 listings
12: 13 listings
13: 4 listings
14: 92 listings
15: 73 listings
16: 10 listings
17: 5 listings
18: 6 listings
19: 4 listings
20: 48 listings
21: 54 listings
22: 7 listings
23: 4 listings
24: 5 listings
25: 21 listings
26: 4 listings
27: 14 listings
28: 61 listings
29: 37 listings
30: 361 listings
31: 81 listings
32: 7 listings
33: 2 listings
34: 4 listings
35: 13 listings
36: 2 listings
37: 1 listings
39: 1 listings
40: 28 listings
41: 1 listings
45: 24 listings
50: 9 listings
55: 2 listings
56: 2 listings
59: 3 listings
60: 132 listings
61: 4 listings
62: 1 listings
64: 3 listings
65: 1 listings
69: 1 listings
70: 2 listings
75: 2 listings
79: 1 listings
80: 4 listings
85: 1 listings
89: 11 listings
90: 168 listings
91: 2 listings
92: 3 listings
93: 1 listings
99: 4 listings
100: 21 listings
103: 1 listings
112: 2 listings

In some fields max nights count is quite weird > 350 days

* minimum_nights

In [21]:
counter = Counter()

for doc in raw_col.find({"minimum_nights": {"$exists": True}}, {"minimum_nights": 1}):
    counter[str(doc["minimum_nights"])] += 1  

for val, count in sorted(counter.items(), key=lambda x: float(x[0].replace("$", "").replace(",", ""))):
    print(f"{val}: {count} listings")

1: 1862 listings
2: 1505 listings
3: 882 listings
4: 296 listings
5: 269 listings
6: 62 listings
7: 178 listings
8: 5 listings
9: 3 listings
10: 38 listings
11: 1 listings
12: 5 listings
13: 2 listings
14: 31 listings
15: 22 listings
16: 2 listings
20: 9 listings
21: 12 listings
22: 1 listings
25: 4 listings
26: 1 listings
27: 3 listings
28: 36 listings
29: 25 listings
30: 116 listings
31: 46 listings
32: 80 listings
35: 1 listings
40: 1 listings
45: 2 listings
50: 1 listings
60: 15 listings
80: 1 listings
90: 12 listings
96: 1 listings
100: 1 listings
115: 1 listings
120: 4 listings
160: 1 listings
179: 1 listings
180: 12 listings
200: 1 listings
300: 1 listings
365: 2 listings
1250: 1 listings


There is one listing with minimum number of nights equivalent to 1250 which can be an error.

* security_deposit

In [22]:
counter = Counter()

for doc in raw_col.find({"security_deposit": {"$exists": True}}, {"security_deposit": 1}):
    value = doc.get("security_deposit")
    if value is not None:
        counter[str(value)] += 1

for val, count in sorted(
    counter.items(),
    key=lambda x: float(x[0].replace("$", "").replace(",", ""))
):
    print(f"{val}: {count} listings")

0.00: 1275 listings
75.00: 1 listings
81.00: 1 listings
85.00: 2 listings
86.00: 1 listings
87.00: 1 listings
88.00: 1 listings
89.00: 1 listings
90.00: 5 listings
95.00: 3 listings
100.00: 201 listings
110.00: 3 listings
120.00: 4 listings
125.00: 6 listings
126.00: 1 listings
128.00: 1 listings
130.00: 4 listings
131.00: 1 listings
132.00: 1 listings
134.00: 1 listings
135.00: 6 listings
136.00: 4 listings
140.00: 5 listings
145.00: 1 listings
150.00: 160 listings
155.00: 1 listings
169.00: 1 listings
170.00: 2 listings
171.00: 1 listings
175.00: 1 listings
180.00: 2 listings
190.00: 2 listings
195.00: 1 listings
197.00: 1 listings
199.00: 6 listings
200.00: 272 listings
206.00: 1 listings
213.00: 1 listings
215.00: 1 listings
240.00: 1 listings
249.00: 1 listings
250.00: 147 listings
265.00: 1 listings
275.00: 3 listings
280.00: 1 listings
299.00: 4 listings
300.00: 209 listings
307.00: 1 listings
309.00: 1 listings
320.00: 1 listings
325.00: 1 listings
329.00: 1 listings
330.00: 3 

Some security deposits seems to be quite high.

* address.country

In [23]:
counter = Counter()

for doc in raw_col.find({"address.country": {"$exists": True}}, {"address.country": 1}):
    market = doc.get("address", {}).get("country")
    if market:
        counter[market.strip()] += 1

for market, count in sorted(counter.items(), key=lambda x: x[0].lower()):
    print(f"{market}: {count} listings")


Australia: 610 listings
Brazil: 606 listings
Canada: 649 listings
China: 19 listings
Hong Kong: 600 listings
Portugal: 555 listings
Spain: 633 listings
Turkey: 661 listings
United States: 1222 listings


* address.market

In [24]:
counter = Counter()

for doc in raw_col.find({"address.market": {"$exists": True}}, {"address.market": 1}):
    market = doc.get("address", {}).get("market")
    if market:
        counter[market.strip()] += 1

for market, count in sorted(counter.items(), key=lambda x: x[0].lower()):
    print(f"{market}: {count} listings")


Barcelona: 632 listings
Hong Kong: 619 listings
Istanbul: 660 listings
Kauai: 67 listings
Maui: 153 listings
Montreal: 648 listings
New York: 607 listings
Oahu: 253 listings
Other (Domestic): 1 listings
Other (International): 4 listings
Porto: 554 listings
Rio De Janeiro: 603 listings
Sydney: 609 listings
The Big Island: 139 listings


address.market containts names of the cities

* address.street

In [25]:
counter = Counter()

for doc in raw_col.find({"address.street": {"$exists": True}}, {"address.street": 1}):
    market = doc.get("address", {}).get("street")
    if market:
        counter[market.strip()] += 1

for market, count in sorted(counter.items(), key=lambda x: x[0].lower()):
    print(f"{market}: {count} listings")


Adalar, Istanbul Province, Turkey: 1 listings
ADALAR, Istanbul, Turkey: 1 listings
Adalar, İstanbul, Turkey: 3 listings
Agrela, Porto, Portugal: 1 listings
Alexandria, NSW, Australia: 5 listings
Alfena, Portugal: 1 listings
Annandale, NSW, Australia: 3 listings
Arcozelo, Vila Nova de Gaia, Porto, Portugal: 1 listings
Arnavutköy, İstanbul, Turkey: 1 listings
Arncliffe, NSW, Australia: 1 listings
Arouca, Aveiro, Portugal: 3 listings
Artarmon, NSW, Australia: 2 listings
Ashfield, NSW, Australia: 2 listings
Astoria, NY, United States: 1 listings
Ataşehir merkez , İstanbul, Turkey: 1 listings
Ataşehir, İstanbul, Turkey: 7 listings
Austin, kowloon, Hong Kong: 1 listings
Avalon Beach, NSW, Australia: 7 listings
Avalon, NSW, Australia: 1 listings
Avcılar, İstanbul, Turkey: 3 listings
Aveiro, Aveiro, Portugal: 3 listings
Bagunte, Porto, Portugal: 1 listings
Bahçelievler, İstanbul, Turkey: 2 listings
Bakırköy, İstanbul, Turkey: 6 listings
Balgowlah, NSW, Australia: 2 listings
Balmain, NSW, Austr

* host_neighbourhood

In [26]:
counter = Counter()

for doc in raw_col.find({"host_neighbourhood": {"$exists": True}}, {"host_neighbourhood": 1}):
    counter[str(doc["host_neighbourhood"])] += 1

for val, count in counter.most_common():
    print(f"{val}: {count} listings")

: 1923 listings
Copacabana: 124 listings
Le Plateau: 123 listings
Mong Kok: 90 listings
Waikiki: 76 listings
Downtown Montreal: 74 listings
Dreta de l'Eixample: 69 listings
Williamsburg: 61 listings
Taksim: 56 listings
Sheung Wan: 55 listings
Tsim Sha Tsui: 55 listings
Sultanahmet: 55 listings
Island of Hawaiʻi: 54 listings
Kihei/Wailea: 51 listings
Ipanema: 48 listings
Kauaʻi: 44 listings
Mile End: 42 listings
Kailua/Kona: 39 listings
Barra da Tijuca: 38 listings
El Raval: 36 listings
Bedford-Stuyvesant: 35 listings
Jordan: 33 listings
Vila de Gràcia: 32 listings
La Petite-Patrie: 32 listings
Şişli: 31 listings
La Sagrada Família: 31 listings
La Nova Esquerra de l'Eixample: 31 listings
Cihangir: 30 listings
L'Antiga Esquerra de l'Eixample: 30 listings
Causeway Bay: 30 listings
Wan Chai: 30 listings
Kennedy Town: 27 listings
Leblon: 26 listings
El Gòtic: 26 listings
Ville-Marie: 26 listings
North Shore: 26 listings
Bushwick: 25 listings
Harlem: 25 listings
Karaköy: 24 listings
Hell's K

---

## Check for missing values/weird values

In [27]:
def get_all_fields(doc, prefix=""):
    fields = set()
    for key, value in doc.items():
        full_key = f"{prefix}.{key}" if prefix else key
        fields.add(full_key)
        if isinstance(value, dict):
            fields.update(get_all_fields(value, full_key))
    return fields

all_fields = set()
for doc in raw_col.find({}, {"_id": 0}):  
    all_fields.update(get_all_fields(doc))

field_counts = []

for field in all_fields:
    query = {
        "$or": [
            {field: {"$exists": False}},
            {field: None},
            {field: ""},
            {field: {}},
            {field: []},
            {field: ''},
            {field: ' '},
        ]
    }
    count = raw_col.count_documents(query)
    
    if count > 0:
        field_counts.append((field, count))

field_counts.sort(key=lambda x: x[1], reverse=True)

for field, count in field_counts:
    print(f"Missing or empty values in '{field}': {count}")


Missing or empty values in 'images.xl_picture_url': 5555
Missing or empty values in 'images.thumbnail_url': 5555
Missing or empty values in 'images.medium_url': 5555
Missing or empty values in 'reviews_per_month': 5461
Missing or empty values in 'monthly_price': 4899
Missing or empty values in 'weekly_price': 4841
Missing or empty values in 'notes': 3080
Missing or empty values in 'interaction': 2478
Missing or empty values in 'access': 2453
Missing or empty values in 'house_rules': 2285
Missing or empty values in 'neighborhood_overview': 2241
Missing or empty values in 'transit': 2232
Missing or empty values in 'host_about': 2219
Missing or empty values in 'security_deposit': 2084
Missing or empty values in 'host_neighbourhood': 1923
Missing or empty values in 'reviews': 1632
Missing or empty values in 'reviews_copy1': 1632
Missing or empty values in 'reviews_copy2': 1632
Missing or empty values in 'reviews_copy4': 1632
Missing or empty values in 'reviews_copy3': 1632
Missing or empty

We can see that fields images.thumbnail_url, images.medium_url, images.xl_picture_url contain only null data so we won't use those fields in further analysis. We could drop them.

* Seting all missing/empty fields to null - normalizing missing/empty values. We decided to change all missing and  non exisitng fields to None (MongoDB missing value) because we want to preserve schema consistency. 

In [28]:
placeholder = None  
update_operations = []

for field, _ in field_counts:
    query = {
        "$or": [
            {field: {"$exists": False}},
            {field: None},
            {field: ""},
            {field: {}},
            {field: []},
            {field: " "},
        ]
    }
    update = {
        "$set": {field: placeholder}
    }

    update_operations.append(UpdateMany(query, update))

if update_operations:
    result = raw_col.bulk_write(update_operations)
    print(f"Updated {result.modified_count} documents to replace missing/empty values with null.")
else:
    print("No missing/empty values found to update.")


Updated 79902 documents to replace missing/empty values with null.


------

## Check for Duplicates

* `host_listings_count` and `host_total_listings_count`: these two fields are duplicated as it was checked above.
<br> While creating a new collection we will include only one of them.

In [29]:
# Count how many documents have matching values
matching = raw_col.count_documents({
    "$expr": {
        "$eq": ["$host_listings_count", "$host_total_listings_count"]
    }
})

# Count total documents where both fields exist
total = raw_col.count_documents({
    "host_listings_count": {"$exists": True},
    "host_total_listings_count": {"$exists": True}
})

print(f"Matching: {matching} / {total} = {matching / total:.2%}")

Matching: 5555 / 5555 = 100.00%


host_listings_count and host_total_listings_count contain the same information. So we decide to not use host_listings_count further while creating the new collection.

* `reviews, reviews_copy1, reviews_copy2, reviews_copy3` and `reviews_copy4`

In [30]:
matching = raw_col.count_documents({
    "$expr": {
        "$and": [
            {"$eq": ["$reviews", "$reviews_copy1"]},
            {"$eq": ["$reviews", "$reviews_copy2"]},
            {"$eq": ["$reviews", "$reviews_copy3"]},
            {"$eq": ["$reviews", "$reviews_copy4"]}
        ]
    }
})

total = raw_col.count_documents({
    "reviews": {"$exists": True},
    "reviews_copy1": {"$exists": True}
})

print(f"Matching: {matching} / {total} = {matching / total:.2%}")

Matching: 5555 / 5555 = 100.00%


Reviews, Reviews_copy1, Reviews_copy2, Reviews_copy3 and Reviews_copy4 contain the same information so we will only use Reviews to avoid data duplication in further analysis 

* `reviews`
<br> We also deleted duplicates inside each listing because some listings contained identical reviews stored multiple times (review copies). We expect improved data consistency and reduced document size. This could benefit query speed and improve storage efficiency. 

In [31]:
# Deduplicate reviews inside each listing
# Key assumption: A review is uniquely identified by (reviewer_id, date, listing_id)

total_duplicates = 0    # Count total removed reviews
deduped_docs = 0        # Count how many listing documents were cleaned

for doc in raw_col.find({"reviews":{"$exists":True, "$ne": None}}, {"_id": 1, "reviews": 1}):
    reviews = doc.get("reviews", [])
    seen_keys = set()           # Store (reviewer_id, date, listing_id)
    unique_reviews = []

    for r in reviews:
        key = (r.get("reviewer_id"), r.get("date"), doc["_id"])

        if key not in seen_keys:
            seen_keys.add(key)
            unique_reviews.append(r)
        else:
            total_duplicates += 1

    # Only update the document if changes are needed
    if len(unique_reviews) < len(reviews):
        deduped_docs += 1
        raw_col.update_one({"_id": doc["_id"]}, {"$set": {"reviews": unique_reviews}})

print(f"Deduplicated reviews in {deduped_docs} listings. Removed {total_duplicates} duplicates.")


Deduplicated reviews in 9 listings. Removed 11 duplicates.


* `amenities`
<br> They are a list of strings like ["TV", "Wifi", "TV"]. We only need each amenity once, so we'll use a set to remove duplicates. We expect reduced noise in filtering and reporting queries.

In [32]:
# Look for a listing that has duplicate entries in its 'amenities' list
for doc in raw_col.find({"amenities": {"$exists": True, "$ne": None}}, {"_id": 1, "name": 1, "amenities": 1}):
    amenities = doc.get("amenities", [])

    # Compare original length with length of set to detect duplicates
    if len(amenities) > len(set(amenities)):
        print(f"Listing ID: {doc['_id']}")
        print(f"Listing Name: {doc.get('name')}")
        print("Original Amenities List:")
        print(amenities)
        break  

Listing ID: 1001265
Listing Name: Ocean View Waikiki Marina w/prkg
Original Amenities List:
['TV', 'Cable TV', 'Wifi', 'Air conditioning', 'Pool', 'Kitchen', 'Free parking on premises', 'Elevator', 'Hot tub', 'Washer', 'Dryer', 'Essentials', 'Shampoo', 'Hangers', 'Hair dryer', 'Iron', 'Laptop friendly workspace', 'Self check-in', 'Lockbox', 'Hot water', 'Bed linens', 'Extra pillows and blankets', 'Ethernet connection', 'Microwave', 'Coffee maker', 'Refrigerator', 'Dishes and silverware', 'Cooking basics', 'Stove', 'BBQ grill', 'Garden or backyard', 'Well-lit path to entrance', 'Disabled parking spot', 'Step-free access', 'Wide clearance to bed', 'Step-free access']


In [33]:
# Amenities are a list of strings like ["TV", "Wifi", "TV"]
# We only need each amenity once, so we'll use a set to remove duplicates

amenities_cleaned = 0

for doc in raw_col.find({"amenities": {"$exists": True, "$ne": None}}, {"_id": 1, "amenities": 1}):
    amenities = doc.get("amenities", [])
    unique_amenities = list(set(amenities))  # Convert to set, then back to list

    # Only update if duplicates were removed
    if len(unique_amenities) < len(amenities):
        amenities_cleaned += 1
        raw_col.update_one({"_id": doc["_id"]}, {"$set": {"amenities": unique_amenities}})

print(f" Deduplicated amenities in {amenities_cleaned} listings.")

 Deduplicated amenities in 483 listings.


*  `host_verifications`
<br> We checked duplicates in the host_verifications field because verification methods should be unique per host. We expect improved data consistency

In [34]:
# Verifications are a list of strings like ["email", "phone"]
# They should be unique per host

verifications_cleaned = 0

for doc in raw_col.find({"host_verifications":{"$exists":True, "$ne":None}}, {"_id": 1, "host_verifications": 1}):
    verifications = doc.get("host_verifications", [])
    unique_verifications = list(set(verifications))

    if len(unique_verifications) < len(verifications):
        verifications_cleaned += 1
        raw_col.update_one({"_id": doc["_id"]}, {"$set": {"host_verifications": unique_verifications}})

print(f"Deduplicated host_verifications in {verifications_cleaned} listings.")

Deduplicated host_verifications in 0 listings.


___

## Errors in the database 

* `host_location`
<br> Host_location has many inconsistencies in the location name, so we decided to create new field host_country which will indicate from which country the host is. But before that we need to change code names such as PT to countries, change all the values to lower case to standarize the size and then filter only the country.

In [35]:
pipeline_high = [
    {"$match": {"host_location": {"$exists": True}}},
    {"$group": {"_id": "$host_location", "frequency": {"$sum": 1}}},
    {"$sort": {"frequency": -1}},  
    {"$limit": 50}
]

print("Top 50 host_locations by frequency:")
for doc in raw_col.aggregate(pipeline_high):
    print(f"{doc['_id']}: {doc['frequency']} listings")


Top 50 host_locations by frequency:
New York, New York, United States: 483 listings
Barcelona, Catalonia, Spain: 407 listings
Montreal, Quebec, Canada: 331 listings
Rio de Janeiro, State of Rio de Janeiro, Brazil: 252 listings
Sydney, New South Wales, Australia: 229 listings
Porto, Porto District, Portugal: 229 listings
Hong Kong: 192 listings
Montreal, Québec, Canada: 161 listings
Honolulu, Hawaii, United States: 130 listings
PT: 129 listings
Istanbul, İstanbul, Turkey: 125 listings
Istanbul, Istanbul, Turkey: 124 listings
US: 116 listings
TR: 111 listings
Hong Kong, Hong Kong: 106 listings
HK: 104 listings
ES: 100 listings
Rio de Janeiro, Rio de Janeiro, Brazil: 97 listings
BR: 96 listings
CA: 87 listings
New South Wales, Australia: 86 listings
Istanbul, Turkey: 80 listings
Barcelona, Cataluña, Spain: 69 listings
Rio, Rio de Janeiro, Brazil: 68 listings
AU: 58 listings
Hawaii, United States: 47 listings
Hong Kong Island, Hong Kong: 34 listings
Brooklyn, New York, United States: 33 li

In [36]:
pipeline_high = [
    {"$match": {"host_location": {"$exists": True}}},
    {"$group": {"_id": "$host_location", "frequency": {"$sum": 1}}},
    {"$sort": {"frequency": 1}}, 
    {"$limit": 50}
]

print("Top 50 host_locations by frequency:")
for doc in raw_col.aggregate(pipeline_high):
    print(f"{doc['_id']}: {doc['frequency']} listings")


Top 50 host_locations by frequency:
Berkeley, California, United States: 1 listings
Dee Why, New South Wales, Australia: 1 listings
Montreux, Vaud, Switzerland: 1 listings
Orinda, California, United States: 1 listings
Okere Falls, Bay Of Plenty, New Zealand: 1 listings
Palmas, Paraná, Brazil: 1 listings
Panama: 1 listings
Mirabel, Québec, Canada: 1 listings
Maricá, Rio de Janeiro, Brazil: 1 listings
c/comte de borrel 115 bis: 1 listings
Hampstead, Quebec, Canada: 1 listings
Arcadia, California, United States: 1 listings
Narwee, New South Wales, Australia: 1 listings
Haines, Alaska, United States: 1 listings
Fatih, Istanbul Province, Turkey: 1 listings
Ropes Crossing, New South Wales, Australia: 1 listings
Burwood, New South Wales, Australia: 1 listings
Paukaa, Hawaii, United States: 1 listings
Mulhouse, Alsace, France: 1 listings
Vancouver, British Columbia, Canada: 1 listings
Reims, Champagne-Ardenne, France: 1 listings
Miami, Florida, United States: 1 listings
Barra de Guaratiba, Sta

In [37]:
pipeline = [
    {"$match": {"host_location": {"$exists": True, "$ne": ""}}},
    {"$project": {
        "clean_location": {
            "$trim": {
                "input": {"$toLower": "$host_location"}
            }
        }
    }},
    {"$group": {
        "_id": "$clean_location",
        "count": {"$sum": 1}
    }},
    {"$sort": {"count": -1}}
]

unique_locations = list(raw_col.aggregate(pipeline))
print(f"Total unique  host_location values: {len(unique_locations)}")


Total unique  host_location values: 665


In [38]:
# filter  the locations which contains only two letters 
pipeline = [
    {
        "$match": {
            "host_location": {
                "$exists": True,
                "$regex": r"^[a-zA-Z]{2}$",  # exactly 2 letters only
                "$options": "i" 
            }
        }
    },
    {
        "$project": {
            "clean_location": {
                "$trim": {
                    "input": {"$toLower": "$host_location"}
                }
            }
        }
    },
    {
        "$group": {
            "_id": "$clean_location",
            "count": {"$sum": 1}
        }
    },
    {"$sort": {"count": -1}}
]

results = list(raw_col.aggregate(pipeline))
for doc in results:
    print(doc)


{'_id': 'pt', 'count': 129}
{'_id': 'us', 'count': 116}
{'_id': 'tr', 'count': 111}
{'_id': 'hk', 'count': 104}
{'_id': 'es', 'count': 100}
{'_id': 'br', 'count': 96}
{'_id': 'ca', 'count': 87}
{'_id': 'au', 'count': 58}
{'_id': 'fr', 'count': 11}
{'_id': 'cn', 'count': 7}
{'_id': 'gb', 'count': 5}
{'_id': 'in', 'count': 3}
{'_id': 'ru', 'count': 3}
{'_id': 'ph', 'count': 2}
{'_id': 'mo', 'count': 1}
{'_id': 'sa', 'count': 1}
{'_id': 'be', 'count': 1}
{'_id': 'uk', 'count': 1}
{'_id': 'th', 'count': 1}
{'_id': 'de', 'count': 1}
{'_id': 'jo', 'count': 1}
{'_id': 'ae', 'count': 1}
{'_id': 'se', 'count': 1}
{'_id': 'tw', 'count': 1}
{'_id': 'ar', 'count': 1}


In [39]:
code_to_country = {
    "pt": "Portugal",
    "us": "United States",
    "tr": "Turkey",
    "hk": "Hong Kong",
    "es": "Spain",
    "br": "Brazil",
    "ca": "Canada",
    "au": "Australia",
    "fr": "France",
    "cn": "China",
    "gb": "United Kingdom",
    "uk": "United Kingdom",
    "ru": "Russia",
    "in": "India",
    "ph": "Philippines",
    "de": "Germany",
    "ae": "United Arab Emirates",
    "tw": "Taiwan",
    "se": "Sweden",
    "jo": "Jordan",
    "be": "Belgium",
    "ar": "Argentina",
    "mo": "Macau",
    "th": "Thailand",
    "sa": "Saudi Arabia",

    # United States
    "us": "United States",
    "usa": "United States",
    "america": "United States",
    "ny": "United States",
    "hi": "United States",
    "hawaii": "United States",
    "california": "United States",
    "new york": "United States",
    "brooklyn": "United States",
    "kauai and colorado": "United States",
    "waikoloa beach resort": "United States",
    "hawaii island": "United States",
    "west maui": "United States",
    "maui": "United States",

    # Canada
    "ca": "Canada",
    "quebec": "Canada",
    "montreal": "Canada",
    "toronto": "Canada",

    # Portugal
    "pt": "Portugal",
    "porto": "Portugal",
    "portugal pt": "Portugal",
    "vila do conde": "Portugal",

    # Spain
    "madrid": "Spain",
    "c/comte de borrel 115 bis": "Spain",

    # Brazil
    "br": "Brazil",
    "brasil": "Brazil",
    "rio": "Brazil",
    "ipanema": "Brazil",
    "rua domingos ferreira": "Brazil",

    # Turkey
    "tr": "Turkey",
    "Istabul":"Turkey",
    "istanbul": "Turkey",
    "i̇stanbul": "Turkey",
    "istanbul türkiye": "Turkey",
    "istanbul/ turkey": "Turkey",
    "çalışıyorum": "Turkey",

    # France
    "paris": "France",

    # Hong Kong / China
    "hk": "Hong Kong",
    "hong kong": "Hong Kong",
    "hongkong": "Hong Kong",
    "hong-kong": "Hong Kong",
    "kowloon": "Hong Kong",
    "九龙": "Hong Kong",
    "中國香港": "Hong Kong",
    "香港": "Hong Kong",
    "sz / hong kong china": "Hong Kong",
    "hong kong/九龙佐敦弥敦道240-252号立信大厦9字楼a室": "Hong Kong",
    "香港九龍旺角花園街": "Hong Kong",
    "新界 - 葵涌 葵涌華星街12號": "Hong Kong",
    "china": "China",
    "中国广东省梅州市梅江区东山大道武警路口金福花园": "China",
    "3/F A6 Pak On Building 105 Austin Road Jordan Kowloon Hong Kong": "Hong Kong",

    # United Kingdom
    "united kingdom": "United Kingdom",
    "london": "United Kingdom",

    # Switzerland
    "switzerland)": "Switzerland",

    # Jordan
    "jordan": "Jordan",
    "amman": "Jordan",

    # UAE
    "united arab emirates": "United Arab Emirates",
    "uae": "United Arab Emirates",

    # South Korea
    "south korea": "South Korea",
    "대한민국": "South Korea",

    # Dominican Republic
    "dominican republic": "Dominican Republic",

    # Democratic Republic of the Congo
    "democratic republic of the congo": "Democratic Republic of the Congo",

}

In [40]:
# Normalize keys (lowercase for consistent matching)
code_to_country = {k.lower(): v for k, v in code_to_country.items()}

def get_country_from_location(location):
    if not location:
        return None
    location = location.lower().strip()

    if location in code_to_country:
        return code_to_country[location]

    for part in location.split(","):
        part = part.strip()
        if part in code_to_country:
            return code_to_country[part]

    for key in code_to_country:
        if key in location:
            return code_to_country[key]

    return None  

updated = 0
for doc in raw_col.find({"host_location": {"$exists": True}}):
    raw_location = doc.get("host_location")
    country = get_country_from_location(raw_location)
    if country:
        raw_col.update_one(
            {"_id": doc["_id"]},
            {"$set": {"host_country": country}}
        )
        updated += 1

print(f"Updated {updated} documents with host_country.")


Updated 5475 documents with host_country.


In [41]:
pipeline_high = [
    {"$match": {"host_country": {"$exists": True}}},
    {"$group": {"_id": "$host_country", "frequency": {"$sum": 1}}},
    {"$sort": {"frequency": -1}}, 
]

for doc in raw_col.aggregate(pipeline_high):
    print(f"{doc['_id']}: {doc['frequency']} listings")


United States: 1724 listings
Canada: 1120 listings
Turkey: 659 listings
Brazil: 589 listings
Hong Kong: 496 listings
Portugal: 415 listings
Spain: 197 listings
Australia: 70 listings
China: 57 listings
France: 45 listings
United Kingdom: 31 listings
India: 28 listings
Argentina: 8 listings
Jordan: 5 listings
Germany: 5 listings
Russia: 4 listings
Sweden: 4 listings
South Korea: 3 listings
United Arab Emirates: 3 listings
Thailand: 2 listings
Philippines: 2 listings
Macau: 2 listings
Taiwan: 1 listings
Belgium: 1 listings
Switzerland: 1 listings
Saudi Arabia: 1 listings
Democratic Republic of the Congo: 1 listings
Dominican Republic: 1 listings


In [42]:
total_countries = raw_col.count_documents({"host_country": {"$ne": None}})
print(f"Number of documents with host_country: {total_countries}")

Number of documents with host_country: 5475


* `invalid dates in transactions and reviews`

In [43]:
# check for any dates that are not datetime
# We will check the reviews and transactions dates

# Check for invalid dates in reviews
invalid_dates = 0
for doc in raw_col.find({"reviews": {"$exists": True, "$ne": None}}, {"_id": 1, "reviews": 1}):
    reviews = doc.get("reviews", [])
    for r in reviews:
        date = r.get("date")
        if not isinstance(date, datetime):  # Check if the date is not a datetime object
            invalid_dates += 1
            print(f"Invalid review date found in listing {doc['_id']}: {date}")
# Check for invalid dates in transactions
invalid_dates = 0
for doc in raw_col.find({"transactions": {"$exists": True, "$ne": None}}, {"_id": 1, "transactions": 1}):
    transactions = doc.get("transactions", {}).get("transactions", [])
    for t in transactions:
        date = t.get("date")
        if not isinstance(date, datetime):  # Check if the date is not a datetime object
            invalid_dates += 1
            print(f"Invalid transaction date found in listing {doc['_id']}: {date}")

In [44]:
cutoff_year = 2007

pre_2007_count = 0                  
total_txns = 0                      
listings_with_old_txns = 0        

# Go through each listing document
for doc in raw_col.find({"transactions": {"$exists": True, "$ne": None}}, {"_id": 1, "transactions.transactions": 1}):
    txns = doc.get("transactions", {}).get("transactions", [])
    has_old_transaction = False    

    for t in txns:
        date = t.get("date")
        if isinstance(date, datetime):  
            total_txns += 1

            if date.year < cutoff_year:
                pre_2007_count += 1
                has_old_transaction = True
 # If this listing had at least one old transaction, count it
    if has_old_transaction:
        listings_with_old_txns += 1

# Print the summary
print(f"Total valid transactions: {total_txns}")
print(f"Transactions before {cutoff_year}: {pre_2007_count}")
print(f"Listings with at least one transaction before {cutoff_year}: {listings_with_old_txns}")

Total valid transactions: 311093
Transactions before 2007: 92374
Listings with at least one transaction before 2007: 5421


There are some trasnactions that are recorded before 2007. Those transactions are likely invalid transactions because Airbnb was started in 2007. 

Any dates before 2007 in transactions must be a product of an error and we should not use them in our analysis before the right dates are established, which is only possible after an investigation by financial department, so for our purposes we decided to drop those transactions.


In [45]:
cutoff_date = datetime(cutoff_year, 1, 1)
updated_count = 0
removed_count = 0

# Now actually remove old transactions
for doc in raw_col.find({"transactions": {"$exists": True, "$ne": None}}, {"_id": 1, "transactions.transactions": 1}):
    txns = doc.get("transactions", {}).get("transactions", [])
    
    # Keep only transactions with date >= 2007
    new_txns = [t for t in txns if isinstance(t.get("date"), datetime) and t["date"] >= cutoff_date]

    # If any were removed, update the document
    if len(new_txns) < len(txns):
        raw_col.update_one(
            {"_id": doc["_id"]},
            {"$set": {"transactions.transactions": new_txns}}
        )
        updated_count += 1
        removed_count += len(txns) - len(new_txns)

print(f"\nRemoved {removed_count} transactions before {cutoff_year} from {updated_count} listings.")


Removed 92374 transactions before 2007 from 5421 listings.


In [46]:
# Define the year Airbnb was founded
cutoff_year = 2007


pre_2007_review_count = 0               
total_valid_reviews = 0               
listings_with_old_reviews = 0       

for doc in raw_col.find({"reviews": {"$ne": None}}, {"_id": 1, "reviews": 1}):
    reviews = doc.get("reviews", [])
    has_old_review = False            

    for r in reviews:
        date = r.get("date")
        if isinstance(date, datetime): 
            total_valid_reviews += 1

            if date.year < cutoff_year:
                pre_2007_review_count += 1
                has_old_review = True

    if has_old_review:
        listings_with_old_reviews += 1

# Print summary results
print(f"Total valid review dates: {total_valid_reviews}")
print(f"Reviews before {cutoff_year}: {pre_2007_review_count}")
print(f"Listings with at least one review before {cutoff_year}: {listings_with_old_reviews}")

Total valid review dates: 149781
Reviews before 2007: 0
Listings with at least one review before 2007: 0


All reviews dates are valid dates

* `Invalid prices` 

In [47]:
invalid_price_count = 0
total_prices_checked = 0
docs_with_invalid_price = 0

for doc in raw_col.find({}, {"price": 1}):
    price = doc.get("price")

    try:
        price_float = float(price)
        total_prices_checked += 1

        if price_float <= 0:
            invalid_price_count += 1
            docs_with_invalid_price += 1
    except (TypeError, ValueError):
        continue  # skip non-numeric or missing prices

# Print results
print(f"Total price fields checked: {total_prices_checked}")
print(f"Invalid prices (<= 0): {invalid_price_count}")
print(f"Documents with invalid prices: {docs_with_invalid_price}")


Total price fields checked: 0
Invalid prices (<= 0): 0
Documents with invalid prices: 0


* `MaximumNights` (for question 6) and `MinimumNights`

It seems like 1125 and 1124 are hardcoded values for maximum_nights or a mistake.  Therefore, we will clean minimum_nights by setting values greater than 365 to null.

It seems like 1862 and 1505 are hardcoded values for minimum_nights or a mistake.  Therefore, we will clean minimum_nights by setting values greater than 365 to null.

In [48]:
updated = 0

for doc in raw_col.find({
    "$or": [
        {"maximum_nights": {"$exists": True}},
        {"minimum_nights": {"$exists": True}}
    ]
}):
    updates = {}

    try:
        max_nights = int(doc.get("maximum_nights", 0))
        if max_nights > 1250:
            updates["maximum_nights"] = None
    except (TypeError, ValueError):
        pass

    try:
        min_nights = int(doc.get("minimum_nights", 0))
        if min_nights > 1250:
            updates["minimum_nights"] = None
    except (TypeError, ValueError):
        pass

    if updates:
        raw_col.update_one({"_id": doc["_id"]}, {"$set": updates})
        updated += 1

print(f"Updated {updated} documents where min/max nights were set to null due to being over {365}.")


Updated 11 documents where min/max nights were set to null due to being over 365.


* `Cities`

In [49]:
# checking the unique values in Address.market field in the listings collection
for location_id in raw_col.distinct('address.market'):
    if location_id is not None:
        count = raw_col.count_documents({'address.market': location_id})
        pprint(f"{location_id}: {count}")

missing_count = raw_col.count_documents({"$or": [
    {"address.market": None},
]})
pprint(f"Missing values: {missing_count}")


'Barcelona: 632'
'Hong Kong: 619'
'Istanbul: 660'
'Kauai: 67'
'Maui: 153'
'Montreal: 648'
'New York: 607'
'Oahu: 253'
'Other (Domestic): 1'
'Other (International): 4'
'Porto: 554'
'Rio De Janeiro: 603'
'Sydney: 609'
'The Big Island: 139'
'Missing values: 6'


We can see that 6 listings have the field address.market with an string value, and 5 listings have non-informative values 'Other (Domestic)' and 'Other (International). We inspect the street field to define to which city they belong.


In [50]:
no_market = raw_col.find({"$or": [{"address.market": None }, {"address.market": "Other (Domestic)"}, {"address.market": "Other (International)"}]},{"address.market": 1, "address.street": 1})
for n in no_market:
    pprint(n)

{'_id': '13363311',
 'address': {'market': None, 'street': 'Waverley, NSW, Australia'}}
{'_id': '13528649',
 'address': {'market': None, 'street': 'New York, NY, United States'}}
{'_id': '13617872',
 'address': {'market': None,
             'street': 'Rio de Janeiro, Rio de Janeiro, Brazil'}}
{'_id': '13702505',
 'address': {'market': None, 'street': 'Montréal, Québec, Canada'}}
{'_id': '14161732',
 'address': {'market': None, 'street': 'Barcelona, Catalunya, Spain'}}
{'_id': '14234514',
 'address': {'market': None, 'street': 'Kapaa, HI, United States'}}
{'_id': '22882443',
 'address': {'market': 'Other (Domestic)',
             'street': 'Maunaloa, HI, United States'}}
{'_id': '25071762',
 'address': {'market': 'Other (International)',
             'street': 'Junqueira, Aveiro, Portugal'}}
{'_id': '27265740',
 'address': {'market': 'Other (International)',
             'street': 'Barra de Guaratiba, Rio de Janeiro, Brazil'}}
{'_id': '30520863',
 'address': {'market': 'Other (Internati

Most of the listings with incorrect Address.market value clearly state the city in Address.field. For the values "Waverley, NSW, Australia", "Kapaa, HI, United States", "Junqueira, Aveiro, Portugal", "Maunaloa, HI, United States" the city information was defined from the territorial belonging information in Wikipedia. We set the values of Address.market to established city denominations.

In [51]:
# street: Waverley, NSW, Australia -> market: Sydney
update_1 = raw_col.update_one({"_id": "13363311"}, {"$set": {"address.market": 'Sydney'}})
pprint(raw_col.find_one({"_id": "13363311"}, {"address.market": 1, "address.street": 1}))

# street: New York, NY, United States -> market: New York
update_2 = raw_col.update_one({"_id": "13528649"}, {"$set": {"address.market": 'New York'}})
pprint(raw_col.find_one({"_id": "13528649"}, {"address.market": 1, "address.street": 1}))

# street: Rio de Janeiro, Rio de Janeiro, Brazil -> market: Rio de Janeiro
update_3 = raw_col.update_one({"_id": "13617872"}, {"$set": {"address.market": 'Rio De Janeiro'}})
pprint(raw_col.find_one({"_id": "13617872"}, {"address.market": 1, "address.street": 1}))

# street: Montréal, Québec, Canada -> market: Montreal
update_4 = raw_col.update_one({"_id": "13702505"}, {"$set": {"address.market": 'Montreal'}})
pprint(raw_col.find_one({"_id": "13702505"}, {"address.market": 1, "address.street": 1}))

# street: Barcelona, Catalunya, Spain -> market: Barcelona
update_5 = raw_col.update_one({"_id": "14161732"}, {"$set": {"address.market": 'Barcelona'}})
pprint(raw_col.find_one({"_id": "14161732"}, {"address.market": 1, "address.street": 1}))

# street: Kapaa, HI, United States -> market: Kauai
update_6 = raw_col.update_one({"_id": "14234514"}, {"$set": {"address.market": 'Kauai'}})
pprint(raw_col.find_one({"_id": "14234514"}, {"address.market": 1, "address.street": 1}))

# street: Junqueira, Aveiro, Portugal -> market: Porto
update_7 = raw_col.update_one({"_id": "25071762"}, {"$set": {"address.market": 'Porto'}})
pprint(raw_col.find_one({"_id": "25071762"}, {"address.market": 1, "address.street": 1}))

# street: Barra de Guaratiba, Rio de Janeiro, Brazil -> market: Rio de Janeiro
update_8 = raw_col.update_one({"_id": "27265740"}, {"$set": {"address.market": 'Rio De Janeiro'}})
pprint(raw_col.find_one({"_id": "27265740"}, {"address.market": 1, "address.street": 1}))

# street: Şile, İstanbul, Turkey -> market: Istanbul
update_9 = raw_col.update_one({"_id": "30520863"}, {"$set": {"address.market": 'Istanbul'}})
pprint(raw_col.find_one({"_id": "30520863"}, {"address.market": 1, "address.street": 1}))

# street: Barra de Guaratiba, Rio de Janeiro, Brazil -> market: Rio de Janeiro
update_10 = raw_col.update_one({"_id": "31989710"}, {"$set": {"address.market": 'Rio De Janeiro'}})
pprint(raw_col.find_one({"_id": "31989710"}, {"address.market": 1, "address.street": 1}))

# street: Maunaloa, HI, United States -> market: Maui
update_11 = raw_col.update_one({"_id": "22882443"}, {"$set": {"address.market": 'Maui'}})
pprint(raw_col.find_one({"_id": "22882443"}, {"address.market": 1, "address.street": 1}))


{'_id': '13363311',
 'address': {'market': 'Sydney', 'street': 'Waverley, NSW, Australia'}}
{'_id': '13528649',
 'address': {'market': 'New York', 'street': 'New York, NY, United States'}}
{'_id': '13617872',
 'address': {'market': 'Rio De Janeiro',
             'street': 'Rio de Janeiro, Rio de Janeiro, Brazil'}}
{'_id': '13702505',
 'address': {'market': 'Montreal', 'street': 'Montréal, Québec, Canada'}}
{'_id': '14161732',
 'address': {'market': 'Barcelona', 'street': 'Barcelona, Catalunya, Spain'}}
{'_id': '14234514',
 'address': {'market': 'Kauai', 'street': 'Kapaa, HI, United States'}}
{'_id': '25071762',
 'address': {'market': 'Porto', 'street': 'Junqueira, Aveiro, Portugal'}}
{'_id': '27265740',
 'address': {'market': 'Rio De Janeiro',
             'street': 'Barra de Guaratiba, Rio de Janeiro, Brazil'}}
{'_id': '30520863',
 'address': {'market': 'Istanbul', 'street': 'Şile, İstanbul, Turkey'}}
{'_id': '31989710',
 'address': {'market': 'Rio De Janeiro',
             'street': 

In [52]:
# checking the unique values in the field Address.market in the listing collection after update
for location_id in raw_col.find().distinct('address.market'):
    pprint(location_id + ': ' + str(raw_col.count_documents({'address.market': location_id})))

# making sure all listings have Address.market field, allowing comprehesive querying on cities
count = raw_col.count_documents({"address.market": {"$exists": True}})
print(f'Number of listings containing Address.market field: {count}')

'Barcelona: 633'
'Hong Kong: 619'
'Istanbul: 661'
'Kauai: 68'
'Maui: 154'
'Montreal: 649'
'New York: 608'
'Oahu: 253'
'Porto: 555'
'Rio De Janeiro: 606'
'Sydney: 610'
'The Big Island: 139'
Number of listings containing Address.market field: 5555


____

### Changing the datatypes

In [53]:
fields_to_check = [
    "bathrooms",
    "cleaning_fee",
    "extra_people",
    "guests_included",
    "monthly_price",
    "price",
    "security_deposit",
    "weekly_price", 
    "host_listing_count",
    "host_response_rate",
    "review_scores_rating",
    "reviews_per_month",
    "minimum_nights",
    "maximum_nights"
]

for field in fields_to_check:
    type_counter = Counter()

    for doc in raw_col.find({}, {field: 1}):
        val = doc.get(field, None)
        type_name = type(val).__name__
        type_counter[type_name] += 1

    print(f"\nField: '{field}' — Value types found:")
    for type_name, count in type_counter.items():
        print(f"  {type_name}: {count} documents")




Field: 'bathrooms' — Value types found:
  Decimal128: 5545 documents
  NoneType: 10 documents

Field: 'cleaning_fee' — Value types found:
  Decimal128: 4024 documents
  NoneType: 1531 documents

Field: 'extra_people' — Value types found:
  Decimal128: 5555 documents

Field: 'guests_included' — Value types found:
  Decimal128: 5555 documents

Field: 'monthly_price' — Value types found:
  NoneType: 4899 documents
  Decimal128: 656 documents

Field: 'price' — Value types found:
  Decimal128: 5555 documents

Field: 'security_deposit' — Value types found:
  Decimal128: 3471 documents
  NoneType: 2084 documents

Field: 'weekly_price' — Value types found:
  NoneType: 4841 documents
  Decimal128: 714 documents

Field: 'host_listing_count' — Value types found:
  NoneType: 5555 documents

Field: 'host_response_rate' — Value types found:
  int: 4167 documents
  NoneType: 1388 documents

Field: 'review_scores_rating' — Value types found:
  int: 4081 documents
  NoneType: 1474 documents

Field: 'r

* Let's set both maximum_nights and minimum_nights to int

In [54]:
updated = 0

for doc in raw_col.find({}, {"_id": 1, "maximum_nights": 1, "minimum_nights": 1}):
    update_fields = {}

    # Convert maximum_nights to int if possible
    try:
        if "maximum_nights" in doc:
            max_nights = int(doc["maximum_nights"])
            update_fields["maximum_nights"] = max_nights
    except (ValueError, TypeError):
        pass

    # Convert minimum_nights to int if possible
    try:
        if "minimum_nights" in doc:
            min_nights = int(doc["minimum_nights"])
            update_fields["minimum_nights"] = min_nights
    except (ValueError, TypeError):
        pass

    # Update document if there are changes to apply
    if update_fields:
        raw_col.update_one({"_id": doc["_id"]}, {"$set": update_fields})
        updated += 1

print(f"Converted maximum_nights and minimum_nights to integers in {updated} documents.")


Converted maximum_nights and minimum_nights to integers in 5555 documents.


* Bathrooms, cleaning_fee, extra_people (can be integer), guests_included (can be integer), monthly_price, price, security deposit, weekly_price are in decimal128 format - we change them to integers/floats

In [55]:
def safe_float(val):
    try:
        if isinstance(val, Decimal128):
            return float(val.to_decimal())
        return float(val)
    except:
        return None

def safe_int(val):
    try:
        if isinstance(val, Decimal128):
            return int(val.to_decimal())
        return int(float(val))
    except:
        return None

float_fields = [
    "bathrooms",
    "cleaning_fee",
    "monthly_price",
    "price",
    "security_deposit",
    "weekly_price"
]

int_fields = [
    "extra_people",
    "guests_included"
]

updated = 0

for doc in raw_col.find({}, {**{f: 1 for f in float_fields + int_fields}, "_id": 1}):
    updates = {}

    for field in float_fields:
        val = doc.get(field)
        converted = safe_float(val)
        if converted is not None:
            updates[field] = converted

    for field in int_fields:
        val = doc.get(field)
        converted = safe_int(val)
        if converted is not None:
            updates[field] = converted

    if updates:
        raw_col.update_one({"_id": doc["_id"]}, {"$set": updates})
        updated += 1

print(f" Updated {updated} documents with cleaned float/int values.")


 Updated 5555 documents with cleaned float/int values.


* check for the type in the nested field - `transactions.price`

In [56]:
unique_types = set()

for doc in raw_col.find({"transactions.transactions.price": {"$exists": True}}, {"transactions.transactions": 1}):
    txns = doc.get("transactions", {}).get("transactions", [])
    for txn in txns:
        price = txn.get("price", None)
        unique_types.add(type(price).__name__)

print("Unique type found in transactions.transactions.price:")
for t in unique_types:
    print(f" {t}")


Unique type found in transactions.transactions.price:
 str


We change those strings to floats and round it to 2 decimal places (now there are numbers with many decimal places and it should be rounded to two decimal places since it's a price)

In [57]:
updated = 0

for doc in raw_col.find({"transactions.transactions.price": {"$exists": True}}, {"transactions.transactions": 1}):
    txns = doc.get("transactions", {}).get("transactions", [])
    new_txns = []
    modified = False

    for txn in txns:
        price = txn.get("price")

        try:
            # Handle long decimal strings → float → round to 2 decimals
            rounded_price = round(float(price), 2)

            # Only update if different from original (as string)
            if str(price) != str(rounded_price):
                txn["price"] = rounded_price
                modified = True
        except (ValueError, TypeError):
            # Leave unconvertible values untouched
            pass

        new_txns.append(txn)

    if modified:
        raw_col.update_one(
            {"_id": doc["_id"]},
            {"$set": {"transactions.transactions": new_txns}}
        )
        updated += 1

print(f"Updated {updated} documents where prices were rounded to 2 decimal places.")


Updated 5531 documents where prices were rounded to 2 decimal places.


---

## Schema Adjustments

### 1. Creating collection `listings`


We applied **the Subset Pattern** to extract only the most relevant fields from the original listingsandReviews documents into the new listings collection. The original documents were large, nested, and included redundant or low-value data (e.g., multiple image sizes, review copies, deeply nested objects).

We normalized host relationships by referencing a separate hosts collection via HostId, and flattened the structure where appropriate (e.g., extracting only picture_url from images). 
We expect improved query performance, reduced document size, and easier indexing and filtering for core attributes like City, Price, and RoomType.
Based on more efficient use of indexes, less memory per document, and cleaner separation of related entities for scalability.



Additionally, in the original database, all ids were strings. We decided to replace them with ObjectIds for efficiency and uniqueness. String ids would require manual id generation and assignment when adding new documents and are more prone to cause duplicates, typos and other inconsistencies. Use of ObjectIds also corresponds to MongoDB standards. While creating ObjectIds, we stored mappings between old string ids and new ObjectIds to ensure reference consistency across entities/collections. It will allow us to maintain compatibility with the old version of the database, e.g. in case of appearence of some old unregistered review which needs to be included, the string ListingId in it can easily be mapped to the relevant ObjectId in the current version of the database. 


In [58]:
# Drop and re-create listings collection
db.listings.drop()
listings_col = db.listings

# initializing id mappings
listing_id_map = {}
host_id_map = {}


migrated_listings = 0

# Iterate through documents in raw_col
for doc in raw_col.find():
    # creating new ObjectIds and mapping them to the old string ids
    old_listing_id = doc["_id"]
    new_listing_id = ObjectId()
    listing_id_map[old_listing_id] = new_listing_id

    old_host_id = doc.get("host_id")
    new_host_id = None
    if old_host_id:
        if old_host_id not in host_id_map:
            host_id_map[old_host_id] = ObjectId()
        new_host_id = host_id_map[old_host_id]


    # Handle review scores
    review_score_fields = {
        "Rating": doc.get("review_scores_rating"),
        "Cleanliness": doc.get("review_scores_cleanliness"),
        "Communication": doc.get("review_scores_communication"),
        "Checkin": doc.get("review_scores_checkin"),
        "Location": doc.get("review_scores_location"),
        "Value": doc.get("review_scores_value")
    }

    reviews = doc.get("reviews")
    if isinstance(reviews, list):
        sample_reviews = [
            {
                "ReviewerName": r.get("reviewer_name"),
                "Date": r.get("date"),
                "Comments": r.get("comments")
            }
            for r in reviews[:3] if isinstance(r, dict)
        ]
    else:
        sample_reviews = None  

    # Build the new listing document
    new_listing = {
        "_id": new_listing_id,
        "Name": doc.get("name"),
        "Address": doc.get("address"),
        "City": doc.get("address", {}).get("market"),  
        "Images": doc.get("images", {}).get("picture_url"), # we only get picture_url, becuase other fields in nested document do not contain any information
        "Price": doc.get("price"),
        "RoomType": doc.get("room_type"),
        "PropertyType": doc.get("property_type"),
        "Beds": doc.get("beds"),
        "Bedrooms": doc.get("bedrooms"),
        "Bathrooms": doc.get("bathrooms"),
        "Accommodates": doc.get("accommodates"),
        "Amenities": doc.get("amenities"),
        "Summary": doc.get("summary"),
        "ReviewScore": review_score_fields,
        "HostId": new_host_id,
        "BedType": doc.get("bed_type"),
        "SecurityDeposit": doc.get("security_deposit"),
        "MaximumNights": doc.get("maximum_nights"),
        "MinimumNights": doc.get("minimum_nights"),
        "ListingURL": doc.get("listing_url"),
        "SampleReviews": sample_reviews 
    }

    # Insert the cleaned and safe listing
    listings_col.insert_one(new_listing)
    migrated_listings += 1

print(f"Migrated {migrated_listings} listings into 'listings' collection.")


Migrated 5555 listings into 'listings' collection.


In [59]:
# Verify listings migration
print("Listings collection contains:", listings_col.count_documents({}), "documents")

# Show one listing to verify structure
sample_listing = listings_col.find_one()
from pprint import pprint
print("Sample migrated listing:")
pprint(sample_listing)

Listings collection contains: 5555 documents
Sample migrated listing:
{'Accommodates': 8,
 'Address': {'country': 'Portugal',
             'country_code': 'PT',
             'government_area': 'Cedofeita, Ildefonso, Sé, Miragaia, Nicolau, '
                                'Vitória',
             'location': {'coordinates': [-8.61308, 41.1413],
                          'is_location_exact': False,
                          'type': 'Point'},
             'market': 'Porto',
             'street': 'Porto, Porto, Portugal',
             'suburb': None},
 'Amenities': ['TV',
               'Cable TV',
               'Wifi',
               'Kitchen',
               'Paid parking off premises',
               'Smoking allowed',
               'Pets allowed',
               'Buzzer/wireless intercom',
               'Heating',
               'Family/kid friendly',
               'Washer',
               'First aid kit',
               'Fire extinguisher',
               'Essentials',
          

---

### 2. Creating collection `hosts`

We applied **the Referencing Pattern** because many hosts appear across multiple listings in the dataset. Duplicating host information (e.g., name, response rate, verification) in every listing would increase document size and create inconsistency if host data changed.

We extracted host data into a separate hosts collection and assigned each host a unique _id. We also removed duplicates and cleaned up repeated fields (e.g., host_verifications or host_listings_count) during the process.

We expect improved data consistency, reduced redundancy, and faster updates to host information.

In [60]:
# migrate hosts listingsAndReviews_new INTO hosts

# Ensure host collection is clean
db.hosts.drop() # Drop the hosts collection if it exists in case you run this multiple times
hosts_col = db.hosts

# Use a set to track host_ids we've already processed
seen_hosts = set()
inserted_count = 0

for doc in raw_col.find({}, {
    "_id": 0,
    "host_id": 1,
    "host_name": 1,
    "host_location": 1,
    "host_is_superhost": 1,
    "host_response_rate": 1,
    "host_response_time": 1,
    "host_total_listings_count": 1,
    # "host_picture_url": 1,
    "host_has_profile_pic": 1,
    "host_identity_verified": 1,
    "host_about": 1,
    "host_url": 1,
    "host_verifications": 1,
    "host_neighbourhood": 1, # not sure if this one should be added (depends what it states)
    "host_country" : 1,

}):
    host_id = doc.get("host_id")

    # Skip hosts with missing string ID or already inserted
    if not host_id or host_id in seen_hosts:
        continue
    # creating new host ObjectId and mapping it to old string id
    if host_id not in host_id_map:
        host_id_map[host_id] = ObjectId()
    
    host_doc = {
    "_id": host_id_map[host_id],
    "Name": doc.get("host_name"),
    "Location": doc.get("host_location"),
    "IsSuperhost": doc.get("host_is_superhost"),
    "ResponseRate": doc.get("host_response_rate"),
    "ResponseTime": doc.get("host_response_time"),
    "TotalListingsCount": doc.get("host_total_listings_count"),
    # "PictureUrl": doc.get("host_picture_url"),
    "HasProfilePic": doc.get("host_has_profile_pic"),
    "IdentityVerified": doc.get("host_identity_verified"),
    "About": doc.get("host_about"),
    "Url": doc.get("host_url"),
    "Verifications": list(set(doc["host_verifications"])) if isinstance(doc.get("host_verifications"), list) else None,
    "Neighbourhood": doc.get("host_neighbourhood"),
    "HostCountry" : doc.get("host_country"),
    }

    # Insert into the hosts collection


    hosts_col.insert_one(host_doc)
    seen_hosts.add(host_id)
    inserted_count += 1

print(f"Inserted {inserted_count} unique hosts into 'hosts' collection.")

Inserted 5104 unique hosts into 'hosts' collection.


In [61]:
# Verify host migration by checking:
# 1. Total count of unique hosts in the new collection
# 2. Sample host document
# 3. Compare with expected number of unique host_ids in the raw data

# Count of hosts in the new collection
new_host_count = hosts_col.count_documents({})
print(f"New 'hosts' collection contains {new_host_count} documents.")

# Get number of unique host_ids in the original collection
unique_host_ids = raw_col.distinct("host_id")
print(f"Original collection contains {len(unique_host_ids)} unique host_ids.")

# Show a sample host from the new collection
sample_host = hosts_col.find_one()
print("Sample migrated host document:")
from pprint import pprint
pprint(sample_host)

New 'hosts' collection contains 5104 documents.
Original collection contains 5104 unique host_ids.
Sample migrated host document:
{'About': 'Gostamos de passear, de viajar, de conhecer pessoas e locais novos, '
          'gostamos de desporto e animais! Vivemos na cidade mais linda do '
          'mundo!!!',
 'HasProfilePic': True,
 'HostCountry': 'Portugal',
 'IdentityVerified': True,
 'IsSuperhost': False,
 'Location': 'Porto, Porto District, Portugal',
 'Name': 'Ana&Gonçalo',
 'Neighbourhood': None,
 'ResponseRate': 100,
 'ResponseTime': 'within an hour',
 'TotalListingsCount': 3,
 'Url': 'https://www.airbnb.com/users/show/51399391',
 'Verifications': ['government_id',
                   'email',
                   'jumio',
                   'phone',
                   'offline_government_id',
                   'reviews'],
 '_id': ObjectId('68054a2843376708823c1713')}


----

### 3. Creating collection `transactions`


We applied **the Referencing Pattern** because transactional data (e.g., price and date of bookings) is nested and potentially unbounded per listing. Embedding all transactions directly in the listings collection would lead to bloated documents and poor performance in historical queries.

We extracted individual transactions into a separate transactions collection, where each document references a ListingId and stores the Date and Price as independent fields.

We expect improved performance for time-based queries and historical pricing analytics, along with smaller listings documents and easier indexing of transaction fields.

In [62]:
# migrate transactions into the new transactions collection

# Drop and re-create transactions collection
db.transactions.drop() # Drop the transactions collection if it exists in case you run this multiple times
transactions_col = db.transactions
migrated_transactions = 0

# Extract transactions per listing
for doc in raw_col.find({}, {"_id": 1, "transactions.transactions": 1}):
    old_id = doc["_id"]
    new_id = listing_id_map.get(old_id)
    if not new_id:
        continue

    txns = doc.get("transactions", {}).get("transactions", [])
    for t in txns:
        try:
            txn_doc = {
                "ListingId": new_id,
                "Date": t.get("date"),
                "Price": float(t.get("price"))
            }
            transactions_col.insert_one(txn_doc)
            migrated_transactions += 1
        except:
            continue  # skip invalid prices
print(f"Migrated {migrated_transactions} transactions into 'transactions' collection.")

Migrated 218719 transactions into 'transactions' collection.


In [63]:
# Check total transactions
print("Transactions collection contains:", transactions_col.count_documents({}), "documents")

# Show one sample transaction
sample_transaction = transactions_col.find_one()
print("Sample migrated transaction:")
pprint(sample_transaction)

Transactions collection contains: 218719 documents
Sample migrated transaction:
{'Date': datetime.datetime(2008, 8, 12, 0, 0),
 'ListingId': ObjectId('68054a2843376708823c1712'),
 'Price': 132.11,
 '_id': ObjectId('68054a4743376708823c40b5')}


---

### 4. Creating collection `reviews`


We applied the Polymorphic and Referencing Pattern because reviews are large, independently queryable, and often repeated within the original listingsandReviews documents. We observed several redundant copies of the reviews (reviews_copy1 to reviews_copy4) that unnecessarily increased document size.

In the transformed schema, we extracted only the main reviews array into a dedicated reviews collection and excluded the redundant copies.
We expect improved performance for review-based queries and analytics, along with smaller, cleaner listing documents and reduced storage overhead. 

In [64]:
# migrate all reviews into new reviews collection

# Drop and re-create reviews collection 
db.reviews.drop()
reviews_col = db.reviews 
migrated_reviews = 0

# initializing mapping of reviewers ids
reviewer_id_map = {}

# Extract all reviews from original listings
for doc in raw_col.find({}, {"_id": 1, "reviews": 1}):
    old_id = doc["_id"]
    new_id = listing_id_map.get(old_id)
    if not new_id:
        continue

    reviews = doc.get("reviews")

    if isinstance(reviews, list):
        for r in reviews:
            if isinstance(r, dict):  # extra safety
                old_reviewer_id = r.get("reviewer_id")

                # creating new reviewers ids and tracking the existing ones
                if old_reviewer_id and old_reviewer_id not in reviewer_id_map:
                    reviewer_id_map[old_reviewer_id] = ObjectId()


                review_doc = {
                    "ListingId": new_id,
                    "ReviewerId": reviewer_id_map.get(old_reviewer_id),
                    "ReviewerName": r.get("reviewer_name"),
                    "Date": r.get("date"),
                    "Comments": r.get("comments")
                }
                reviews_col.insert_one(review_doc)
                migrated_reviews += 1

print(f"Migrated {migrated_reviews} reviews into 'reviews' collection.")


Migrated 149781 reviews into 'reviews' collection.


In [65]:
# Check total reviews
print("Reviews collection contains:", reviews_col.count_documents({}), "documents")

# Show one sample review
sample_review = reviews_col.find_one()
print("Sample migrated review:")
pprint(sample_review)

Reviews collection contains: 149781 documents
Sample migrated review:
{'Comments': 'A casa da Ana e do Gonçalo foram o local escolhido para a '
             'passagem de ano com um grupo de amigos. Fomos super bem '
             'recebidos com uma grande simpatia e predisposição a ajudar com '
             'qualquer coisa que fosse necessário.\r\n'
             'A casa era ainda melhor do que parecia nas fotos, totalmente '
             'equipada, com mantas, aquecedor e tudo o que pudessemos '
             'precisar.\r\n'
             'A localização não podia ser melhor! Não há melhor do que acordar '
             'de manhã e ao virar da esquina estar a ribeira do Porto.',
 'Date': datetime.datetime(2016, 1, 3, 5, 0),
 'ListingId': ObjectId('68054a2843376708823c1712'),
 'ReviewerId': ObjectId('68054ba143376708823f9714'),
 'ReviewerName': 'Cátia',
 '_id': ObjectId('68054ba143376708823f9715')}


___

### Final check of all collections

In [66]:
collection_list = db.list_collection_names()

print(f"The database contains {len(collection_list)} collections")
print(f"All collections: {collection_list[0:]}")

The database contains 5 collections
All collections: ['listingsAndReviews_new', 'transactions', 'listings', 'hosts', 'reviews']


____

## Creating indexes

`listings Collection`
<br> Indexes for each question: 
- Filter by City, RoomType, PropertyType, BedType (Q3, Q6, Q11)
- Sort by or group by Price, ReviewScore.Rating, SecurityDeposit (Q5, Q11)
- Access listings by HostId (Q2)
- Search by Amenities or Summary (Q3, Q7)

In [67]:
db.listings.create_index("City")
db.listings.create_index("RoomType")
db.listings.create_index("PropertyType")
db.listings.create_index("BedType")
db.listings.create_index("HostId")
db.listings.create_index("Price")
db.listings.create_index("SecurityDeposit")
db.listings.create_index("ReviewScore.Rating")
db.listings.create_index("Amenities")  # supports inclusion search

'Amenities_1'

Listings collection indexes 
- **City** - We applied an index on City because it is commonly used in search filters. We expect faster query performance when users filter listings by city.

- **RoomType, PropertyType, BedType** - We applied indexes on these categorical fields because they support common filtering options on the platform. 

- **HostId** - We applied this index to support efficient joins between listings and hosts. We expect faster retrieval of all listings by a host, based on indexed lookups for relational mapping.

- **Price** - We applied this index because price is frequently used in range queries. We expect improved sorting and filtering performance based on range index usage.

- **SecurityDeposit**  - We indexed this to support price transparency features and filtering for listings that require deposits.

- **ReviewScore.Rating** - We indexed the nested ReviewScore.Rating field because users often sort listings by rating. 

- **Amenities** - We applied this index to support filtering (e.g., has "Wifi", "Kitchen"). We expect improved query speed on amenities-based searches



`hosts Collection`
<br> Indexes useful in questions: 
- IsSuperhost - Superhosts with multiple listings (Q2)
- TotalListingsCount - Joining with listings

In [68]:
db.hosts.create_index("IsSuperhost")
db.hosts.create_index("TotalListingsCount")


'TotalListingsCount_1'

hosts Collection indexes 
- **IsSuperhost** - We applied this index to support filtering hosts by badge type. 

- **TotalListingsCount** - We indexed this to support sorting and filtering hosts by their activity level. 

<br> We expect better performance in analytics dashboards or host-based segmentation queries.

`reviews Collection`
Indexes useful in questions: 
- Find longest reviews (Q4) 
- Top 20 reviewers, reviews per reviewer (Q8)
- Inserting review for new property (Q13)

In [69]:
db.reviews.create_index("ListingId")
db.reviews.create_index("ReviewerId")
db.reviews.create_index("ReviewerName")
db.reviews.create_index([("ReviewerId", 1), ("Date", -1)])  # for sorting and recency

'ReviewerId_1_Date_-1'

reviews Collection indexes
- **ListingId** - We applied this index to support joins back to listings and to retrieve all reviews per listing efficiently.

- **ReviewerId** - We indexed this to enable queries like seeing all reviews made by a user. We expect fast lookups when analyzing reviewer activity.

- **ReviewerName**  - We applied this to support text search or filtering by user name. 

- **(ReviewerId, Date)** - We applied this compound index to support queries that retrieve a reviewer’s latest reviews. 

`transactions Collection`
Indexes used for: 
- average value of transactions over time for a given property (Q10)

In [70]:
db.transactions.create_index("Listing_id")
db.transactions.create_index("Date")  # supports time range filtering
db.transactions.create_index([("ListingId", 1), ("Date", -1)])

'ListingId_1_Date_-1'

transactions Collection indexes
- **ListingId** - We indexed this to link transaction records back to listings. We expect faster joins and aggregation in booking/availability analytics.

- **Date** - We indexed this field to support time-based filtering (e.g., show bookings in the last month). We expect improvements in performance for date range reports and trend analysis.

- **(ListingId, Date)** - We applied this compound index to efficiently retrieve the transaction history of a listing, ordered by date. We expect improved query speed for calendar views and historical pricing.

----

<font size="6">2. Uses for the database</font>

2)	Once a month, we reward hosts with recognition. Select three superhosts with at least two listings that can accommodate more than four people.

In [71]:
# getting listings that can accommodate more than four people
query_1 = {"$match": {"Accommodates": {"$gt": 4}}}

# joining with host collection on hostId/_id
query_2 = {
        "$lookup": {
            "from": "hosts",
            "localField": "HostId",
            "foreignField": "_id",
            "as": "host"
        }
    }

# unwinding to access host fields
query_3 = {"$unwind": "$host"}

# filtering out only superhosts
query_4 = {"$match": {"host.IsSuperhost": True}}

# grouping by host to count listings
query_5 = {
        "$group": {
            "_id": "$HostId",
            "host_name": {"$first": "$host.Name"},
            "listing_count": { "$count": {} }
        }
    }

# filtering hosts with at least 2 listings
query_6 = {"$match": {"listing_count": {"$gte": 2}}}

# optional: sorting in descending order: assume number of qualifying listing as selection criteria for hosts to be rewarede
query_7 = {"$sort": {"listing_count" : -1}}

# limmiting to 3 superhosts
query_8 = {"$limit": 3}


pipeline = [query_1, query_2, query_3, query_4, query_5, query_6, query_7, query_8]

result = listings_col.aggregate(pipeline)

for r in result:
    pprint(r)

{'_id': ObjectId('68054a2a43376708823c1982'),
 'host_name': 'Great Vacation Retreats',
 'listing_count': 3}
{'_id': ObjectId('68054a2a43376708823c19db'),
 'host_name': 'Patty And Beckett',
 'listing_count': 3}
{'_id': ObjectId('68054a2d43376708823c1f61'),
 'host_name': 'Elite',
 'listing_count': 3}


3)	The company considers investing into property to rent. Which bed type is most common in listings with a waterfront and a dishwasher in New York?

In [72]:
# getting the listings which have both waterfront and a dishwasher in New York
query_1 = {"$match" : {"$and": [
        {"Amenities": {"$all": ["Waterfront", "Dishwasher"]}},  
        {"City": "New York"} 
    ]
} }

#grouping by bed type
query_2 = {"$group" : {"_id": "$BedType", "sum": { "$sum": 1}}}

# sorting in descending order
query_3 = {"$sort": {"sum" : -1}}

# limiting to 1 most common type
query_4 = {"$limit": 1}

pipeline = [query_1, query_2, query_3, query_4]

result = listings_col.aggregate(pipeline)

for r in result:
    pprint(r)

{'_id': 'Real Bed', 'sum': 1}


4)	We're considering hiring someone to write reviews professionally. Who wrote the longest review in New York?

In [73]:
pipeline = [
    {
        "$lookup": {
            "from": "listings",
            "localField": "ListingId",
            "foreignField": "_id",
            "as": "listing"
        }
    },
    { "$unwind": "$listing" },

    {
        "$match": {
            "listing.Address.market": "New York",
            "Comments": { "$type": "string", "$ne": "" }
        }
    },

    {
        "$addFields": {
            "review_length": { "$strLenCP": "$Comments" }
        }
    },

    { "$sort": { "review_length": -1 } },

    { "$limit": 1 },

    {
        "$project": {
            "_id": 0,
            "Reviewer": "$ReviewerName",
            "Review": "$Comments",
            "Length": "$review_length",
            "ListingId": 1  
        }
    }
]

result = list(db.reviews.aggregate(pipeline))
pprint(result)


[{'Length': 4665,
  'ListingId': ObjectId('68054a3b43376708823c3ec6'),
  'Review': 'This alleged “full bedroom”; Private room with the red couch '
            'shown on the listing’s pictures (which the couch is not in the '
            'room, at least it wasn’t when I arrived) is a complete no go for '
            'me. 5 stars DOWN. This was a booking that I made less than 48 '
            'hours from my scheduled arrival. I have been using this AIRBNB '
            'service now for 3 years, so I would like to think I’m pretty '
            'familiar with how things work. The raised eyebrows began when '
            'Carlos sent me a text message outside of AirBnB directly from a '
            'cell phone number to my cell phone number the day of check in '
            'requesting that I send him a picture of my US ID or Passport, as '
            'it is needed for self-check in. Again, since I have reserved with '
            'several hosts in the past who only did self-check in and 

5)	To assess the security of different areas, what is the biggest and smallest (price-security deposit) difference per number of visitors at a property?

In [ ]:
# as area, we will inspect the cities. for each city we will fin the minimal and maximal difference between
# price and security deposit by number of visitors

pipeline = [
    {
        "$match": {
            "Price": { "$type": "double" },
            "SecurityDeposit": { "$type": "double" }
        }
    },
    {
        "$project": {
            "Accommodates": 1,
            "City": 1,
            "Difference": { "$subtract": ["$Price", "$SecurityDeposit"] }
        }
    },
    {
        "$group": {
            "_id": {'City': "$City", "number_of_visitors": "$Accommodates"},
            "MaxDiff": { "$max": "$Difference" },
            "MinDiff": { "$min": "$Difference" }
        }
    },
    {
        "$project": {
            "_id": 0,
            "City": '$_id.City',
            "Visitors": '$_id.number_of_visitors',
            "MaxDiff": 1,
            "MinDiff": 1 }
    },
    { "$sort": {"City":1, "Visitors":1} }
]

results = list(db.listings.aggregate(pipeline))
results


[{'MaxDiff': 60.0, 'MinDiff': -1960.0, 'City': 'Barcelona', 'Visitors': 1},
 {'MaxDiff': 110.0, 'MinDiff': -4373.0, 'City': 'Barcelona', 'Visitors': 2},
 {'MaxDiff': 85.0, 'MinDiff': -2185.0, 'City': 'Barcelona', 'Visitors': 3},
 {'MaxDiff': 2700.0, 'MinDiff': -3385.0, 'City': 'Barcelona', 'Visitors': 4},
 {'MaxDiff': 100.0, 'MinDiff': -1931.0, 'City': 'Barcelona', 'Visitors': 5},
 {'MaxDiff': 350.0, 'MinDiff': -2221.0, 'City': 'Barcelona', 'Visitors': 6},
 {'MaxDiff': 500.0, 'MinDiff': -192.0, 'City': 'Barcelona', 'Visitors': 7},
 {'MaxDiff': 740.0, 'MinDiff': -211.0, 'City': 'Barcelona', 'Visitors': 8},
 {'MaxDiff': -171.0, 'MinDiff': -171.0, 'City': 'Barcelona', 'Visitors': 9},
 {'MaxDiff': 400.0, 'MinDiff': -440.0, 'City': 'Barcelona', 'Visitors': 10},
 {'MaxDiff': -125.0, 'MinDiff': -125.0, 'City': 'Barcelona', 'Visitors': 11},
 {'MaxDiff': 0.0, 'MinDiff': 0.0, 'City': 'Barcelona', 'Visitors': 12},
 {'MaxDiff': -700.0, 'MinDiff': -700.0, 'City': 'Barcelona', 'Visitors': 13},
 {'Ma

6)  Identify areas by whether they are typically used for short breaks, like weekend mini breaks, or whether they are more suitable for long trips. This information support targeted advertising of different customer types. It is not expected to change much over time so we won’t look to update it, we just require current view. What is the average duration of stay (in nights) per type of property per city (you can use the maximum_nights to measure length of stays)? For each property type return the city with the highest and lowest average value.

In [75]:
pipeline = [
    {
        "$match": {
            "MaximumNights": { "$type": "int" },
            "City": { "$ne": None },
            "PropertyType": { "$ne": None }
        }
    },
    {
        "$group": {
            "_id": { "PropertyType": "$PropertyType", "City": "$City" },
            "AvgNights": { "$avg": "$MaximumNights" }
        }
    },
    {
        "$project": {
            "_id": 1,
            "RoundedAvg": { "$round": ["$AvgNights", 0] } 
        }
    },
    {
        "$sort": {
            "_id.PropertyType": 1,
            "RoundedAvg": 1
        }
    },
    {
        "$group": {
            "_id": "$_id.PropertyType",
            "Lowest": { "$first": { "City": "$_id.City", "Avg": "$RoundedAvg" } },
            "Highest": { "$last": { "City": "$_id.City", "Avg": "$RoundedAvg" } }
        }
    },
    {
        "$sort": { "_id": 1 }
    }
]

In [76]:
def classify_duration(avg_nights):
    if avg_nights <= 7:
        return "Short stay"
    elif avg_nights <= 30:
        return "Medium stay"
    else:
        return "Long stay"
 

results = list(db.listings.aggregate(pipeline))
for item in results:
    prop_type = item["_id"]
    low = item["Lowest"]
    high = item["Highest"]

    print(f"Property Type: {prop_type}")
    print(f" City with the lowest Avgerage Stay: {low['City']} ({int(low['Avg'])} nights) - {classify_duration(low['Avg'])}")
    print(f" City with the highest Avgerage Stay: {high['City']} ({int(high['Avg'])} nights) - {classify_duration(high['Avg'])}\n")



Property Type: Aparthotel
 City with the lowest Avgerage Stay: Oahu (60 nights) - Long stay
 City with the highest Avgerage Stay: Montreal (1125 nights) - Long stay

Property Type: Apartment
 City with the lowest Avgerage Stay: The Big Island (371 nights) - Long stay
 City with the highest Avgerage Stay: Kauai (833 nights) - Long stay

Property Type: Barn
 City with the lowest Avgerage Stay: The Big Island (1125 nights) - Long stay
 City with the highest Avgerage Stay: The Big Island (1125 nights) - Long stay

Property Type: Bed and breakfast
 City with the lowest Avgerage Stay: Kauai (30 nights) - Medium stay
 City with the highest Avgerage Stay: Oahu (1125 nights) - Long stay

Property Type: Boat
 City with the lowest Avgerage Stay: Barcelona (1125 nights) - Long stay
 City with the highest Avgerage Stay: Porto (1125 nights) - Long stay

Property Type: Boutique hotel
 City with the lowest Avgerage Stay: Sydney (578 nights) - Long stay
 City with the highest Avgerage Stay: Rio De Jane

7)	We are creating a new webpage for hosts when setting up their account. It will list suggested typical amenities. This data will need to be available every time a host registers a property but is not expected to change very much. The starting point for the list will be all unique amenities currently listed in properties (across all documents). Optimise the database for this use case and show how the data should be queried.

We created an amenities collection populated with unique values from existing listings, stored with a compound index on recommended and name. This enables fast and reliable lookup of suggested amenities for hosts when setting up a property. Since this data changes infrequently, we decoupled it from the listings collection to reduce load and improve frontend performance.

In [77]:
# Drop old amenities collection (if re-running)
db.amenities.drop()
amenities_col = db.amenities

pipeline = [
    { "$unwind": "$amenities" },
    { "$match": { "amenities": { "$ne": None } } },
    { "$group": { "_id": "$amenities" } },
    { "$project": { "_id": 0, "name": "$_id" } }
    
]

unique_amenities = list(raw_col.aggregate(pipeline))

# Insert into new amenities collection
for amenity in unique_amenities:
    name = amenity["name"]
    doc = {
        "_id": name.lower().replace(" ", "_"),
        "name": name
    }
    amenities_col.insert_one(doc)

In [78]:
print("Amenities collection contains:", amenities_col.count_documents({}), "documents")

Amenities collection contains: 185 documents


In [79]:
all_amenities = db.amenities.find(
    {},
    { "_id": 0, "name": 1 }
).sort("name", 1)

print("All Unique Amenities:")
for a in all_amenities:
    print("-", a["name"])


All Unique Amenities:
- 24-hour check-in
- Accessible-height bed
- Accessible-height toilet
- Air conditioning
- Air purifier
- Alfresco shower
- BBQ grill
- Baby bath
- Baby monitor
- Babysitter recommendations
- Balcony
- Bath towel
- Bathroom essentials
- Bathtub
- Bathtub with bath chair
- Beach chairs
- Beach essentials
- Beach view
- Beachfront
- Bed linens
- Bedroom comforts
- Bicycle
- Bidet
- Body soap
- Boogie boards
- Breakfast
- Breakfast bar
- Breakfast table
- Building staff
- Buzzer/wireless intercom
- Cable TV
- Carbon monoxide detector
- Cat(s)
- Ceiling fan
- Central air conditioning
- Changing table
- Chef's kitchen
- Children’s books and toys
- Children’s dinnerware
- Cleaning before checkout
- Coffee maker
- Convection oven
- Cooking basics
- Crib
- DVD player
- Day bed
- Dining area
- Disabled parking spot
- Dishes and silverware
- Dishwasher
- Dog(s)
- Doorman
- Double oven
- Dryer
- EV charger
- Electric profiling bed
- Elevator
- En suite bathroom
- Espresso ma

There are two amenities that are likely missing values/erros
- translation missing: en.hosting_amenity_49
- translation missing: en.hosting_amenity_50. 
<br> Thus we delete these broken values from all documents.

In [80]:
db.amenities.delete_many({
    "name": { "$in": ["translation missing: en.hosting_amenity_49", "translation missing: en.hosting_amenity_50"]}
})


DeleteResult({'n': 2, 'ok': 1.0}, acknowledged=True)

In [81]:
all_amenities = db.amenities.find(
    {},
    { "_id": 0, "name": 1 }
).sort("name", 1)

print("All Unique Amenities:")
for a in all_amenities:
    print("-", a["name"])


All Unique Amenities:
- 24-hour check-in
- Accessible-height bed
- Accessible-height toilet
- Air conditioning
- Air purifier
- Alfresco shower
- BBQ grill
- Baby bath
- Baby monitor
- Babysitter recommendations
- Balcony
- Bath towel
- Bathroom essentials
- Bathtub
- Bathtub with bath chair
- Beach chairs
- Beach essentials
- Beach view
- Beachfront
- Bed linens
- Bedroom comforts
- Bicycle
- Bidet
- Body soap
- Boogie boards
- Breakfast
- Breakfast bar
- Breakfast table
- Building staff
- Buzzer/wireless intercom
- Cable TV
- Carbon monoxide detector
- Cat(s)
- Ceiling fan
- Central air conditioning
- Changing table
- Chef's kitchen
- Children’s books and toys
- Children’s dinnerware
- Cleaning before checkout
- Coffee maker
- Convection oven
- Cooking basics
- Crib
- DVD player
- Day bed
- Dining area
- Disabled parking spot
- Dishes and silverware
- Dishwasher
- Dog(s)
- Doorman
- Double oven
- Dryer
- EV charger
- Electric profiling bed
- Elevator
- En suite bathroom
- Espresso ma

8)	We plan to track our reviewers better. We want to create a webpage that shows the top 20 reviewers and the count of the number of reviews of each of these reviewers. This webpage should be kept up to date. It should also have a link to return the number of reviews for a given reviewer ID or Name (show how to query for number of reviews by ID or query quickly).


In this task we use the previously created collection reviews. And we gonna use indexes: ReviewerId and ReviewerName.

In [82]:
# Get top 20 reviewers for the webpage
# we dont create new collection out of it becuase the data can be change often
pipeline = [
    {
        "$group": {
            "_id": "$ReviewerId",
            "Name": { "$first": "$ReviewerName" },
            "ReviewCount": { "$sum": 1 }
        }
    },
    { "$sort": { "ReviewCount": -1 } },
    { "$limit": 20 }
]

top_reviewers = list(db.reviews.aggregate(pipeline))

for reviewer in top_reviewers:
    print(f"{reviewer['Name']} (ID: {reviewer['_id']}) – {reviewer['ReviewCount']} reviews")


Filipe (ID: 68054bcb4337670882405cd0) – 24 reviews
Nick (ID: 68054c1e433767088241f06f) – 13 reviews
Uge (ID: 68054c034337670882416545) – 10 reviews
Thien (ID: 68054bcd43376708824066ad) – 9 reviews
Lisa (ID: 68054c3d4337670882428a06) – 9 reviews
David (ID: 68054bb443376708823fe774) – 8 reviews
Todd (ID: 68054bf64337670882411e76) – 8 reviews
Courtney (ID: 68054bd443376708824083d6) – 8 reviews
Jodi (ID: 68054c634337670882433a27) – 8 reviews
Lisa (ID: 68054c1e433767088241f067) – 8 reviews
Lance (ID: 68054bb143376708823fdcb6) – 7 reviews
David (ID: 68054c6543376708824341f5) – 7 reviews
Gonzalo (ID: 68054c2b4337670882422655) – 6 reviews
Chris (ID: 68054c7b433767088243aebd) – 6 reviews
Dan (ID: 68054bef433767088240f4b5) – 6 reviews
Pierre (ID: 68054bc0433767088240225a) – 6 reviews
Mary (ID: 68054bff4337670882414cb3) – 6 reviews
Karen (ID: 68054bc64337670882404360) – 6 reviews
Megan (ID: 68054bf443376708824111ee) – 6 reviews
Assis (ID: 68054c604337670882433516) – 6 reviews


Quries to support fast lookup

In [83]:
# By ID (using index ReviewerID)
reviewer_id = db.reviews.find_one({'ReviewerName': 'Thien'}, {"ReviewerId": 1}).get("ReviewerId")
by_id = db.reviews.count_documents({ "ReviewerId": reviewer_id })
print(f"Reviewer with ID: {reviewer_id} wrote {by_id} reviews")

Reviewer with ID: 68054bcd43376708824066ad wrote 9 reviews


In [84]:
# By Name (using index ReviewerName) -  we also ensure that our query is case-insensitive
reviewer_name = "Nick"
by_name = db.reviews.count_documents({ "ReviewerName": { "$regex": f"^{reviewer_name}$", "$options": "i" } })
print(f"{reviewer_name} wrote {by_name} reviews.")

Nick wrote 265 reviews.


In [85]:
unique_name = db.reviews.distinct("ReviewerId", { "ReviewerName": "Nick" })
print("Number of different reviewers named Nick:", len(unique_name))

Number of different reviewers named Nick: 247


While counting number of reviews only by name returns for specific name different number of reviews for name Nick than it is listed in top20 reviews and that is because there is 247 different users named Nick, so it is better to filter based on ID since this field is unique. 

9)	For each property we store review scores across different metrics (accuracy, check-in, cleanliness etc). We consider adding more metrics, although there is no clarity on what these will be. We want to be able to easily query the average score across all of these metrics, including any new metrics that might be added without changing the query. Adjust the data model so this can be done and show the query for an example property.


Originally, review scores were stored as separate fields. During creation of new lsitings we consolidated them into a nested ReviewScore document. So it supports dynamic aggregation now. This allows to add new metrics without changing the query. Using $objectToArray and $avg, we can calculate the average review score across all metrics. 

The only exception which will not be taken into account is the metric Rating, since, unlike all other metrics ranging between 0 and 10, it ranges between 0 and 100, which will distort the mean value, and does not appear to come directly from the users reviews. It may be some more fine-grained aggregate metric which takes into account not only reviews, but also other operational factors, so will keep it in ReviewScore for other applications, but not include in the calculation of the average.

In [86]:
sample_doc = db.listings.find_one()
pprint(sample_doc)
sample_doc_id = sample_doc.get('_id') #for further test

{'Accommodates': 8,
 'Address': {'country': 'Portugal',
             'country_code': 'PT',
             'government_area': 'Cedofeita, Ildefonso, Sé, Miragaia, Nicolau, '
                                'Vitória',
             'location': {'coordinates': [-8.61308, 41.1413],
                          'is_location_exact': False,
                          'type': 'Point'},
             'market': 'Porto',
             'street': 'Porto, Porto, Portugal',
             'suburb': None},
 'Amenities': ['TV',
               'Cable TV',
               'Wifi',
               'Kitchen',
               'Paid parking off premises',
               'Smoking allowed',
               'Pets allowed',
               'Buzzer/wireless intercom',
               'Heating',
               'Family/kid friendly',
               'Washer',
               'First aid kit',
               'Fire extinguisher',
               'Essentials',
               'Hangers',
               'Hair dryer',
               'Iron',
  

We implemented a function get_average_review_score(listing_id) that dynamically calculates the average of all review metrics for a given property. 

In [87]:
def get_average_review_score(listing_id: str):
    pipeline = [
        {
            "$match": { "_id": listing_id }
        },
        {
        "$project": {
            "ReviewScore": 1,
            "score_array": {
                "$filter": {
                    "input": { "$objectToArray": "$ReviewScore" },
                    "as": "item",
                    "cond": { "$ne": ["$$item.k", "Rating"] }
                    }
                }
            }
        },
        {
            "$project": {
                "average_score": {
                    "$round": [{ "$avg": "$score_array.v" }, 2]
                },
                "ReviewScore": 1
            }
        }
    ]

    result = list(db.listings.aggregate(pipeline))
    return result[0] if result else None

In [88]:
# Example use
result = get_average_review_score(sample_doc_id)
pprint(result)

{'ReviewScore': {'Checkin': 10,
                 'Cleanliness': 9,
                 'Communication': 10,
                 'Location': 10,
                 'Rating': 89,
                 'Value': 9},
 '_id': ObjectId('68054a2843376708823c1712'),
 'average_score': 9.6}


10)	We aim to have better access to information about transaction, we wish to develop a search engine that can calculate the average value of transactions in a given period of time quickly for a given property.


To support fast, time-filtered transaction analysis per property, we created a compound index on ("ListingId", "Date"). This allows MongoDB to efficiently filter documents for a given property within a date range. We use a simple aggregation pipeline with $match and $group to calculate the average Price. This design enables the backend of a search engine or reporting system to retrieve per-property transaction averages in milliseconds, even on large datasets.



In [89]:
sample_doc = db.transactions.find_one()
pprint(sample_doc)
sample_tx_listing = sample_doc.get('ListingId') # for further test

{'Date': datetime.datetime(2008, 8, 12, 0, 0),
 'ListingId': ObjectId('68054a2843376708823c1712'),
 'Price': 132.11,
 '_id': ObjectId('68054a4743376708823c40b5')}


In [90]:
pipeline = [
    {
        "$group": {
            "_id": "$ListingId",
            "count": { "$sum": 1 }
        }
    },
    {
        "$sort": { "count": -1 }  # Optional: sort by most common
    }
]

results = list(db.transactions.aggregate(pipeline))
for doc in results:
    print(f"{doc['_id']}: {doc['count']} transactions")


68054a2a43376708823c19bb: 98 transactions
68054a2c43376708823c1c8a: 96 transactions
68054a3243376708823c2962: 96 transactions
68054a3943376708823c3c27: 96 transactions
68054a3343376708823c2d8b: 96 transactions
68054a3643376708823c36d7: 96 transactions
68054a2f43376708823c227f: 96 transactions
68054a3343376708823c2ead: 96 transactions
68054a3b43376708823c3e63: 96 transactions
68054a3743376708823c37b8: 96 transactions
68054a2f43376708823c22db: 96 transactions
68054a3043376708823c2533: 94 transactions
68054a2f43376708823c2374: 94 transactions
68054a2d43376708823c1de2: 94 transactions
68054a3343376708823c2d0b: 94 transactions
68054a3743376708823c3878: 94 transactions
68054a3943376708823c3af6: 94 transactions
68054a2c43376708823c1c5c: 88 transactions
68054a3443376708823c3078: 88 transactions
68054a3543376708823c32cd: 88 transactions
68054a2b43376708823c1b97: 87 transactions
68054a3943376708823c3c78: 87 transactions
68054a3343376708823c2dbb: 87 transactions
68054a2a43376708823c1846: 86 trans

We implemented a function called get_average_transaction_price() that acts as the core of a search engine to compute average transaction prices per property over time. This function accepts a listing ID and a date range and returns the average price and transaction count. 

In [91]:
def get_average_transaction_price(listing_id: str, start_date: datetime, end_date: datetime):
    pipeline = [
        {
            "$match": {
                "ListingId": listing_id,
                "Date": { "$gte": start_date, "$lte": end_date }
            }
        },
        {
            "$group": {
                "_id": "$ListingId",
                "avg_price_raw": { "$avg": "$Price" },
                "transaction_count": { "$sum": 1 }
            }
        },
        {
            "$project": {
                "_id": 1,
                "average_price": { "$round": ["$avg_price_raw", 2] },
                "transaction_count": 1
            }
        }
    ]

    result = list(db.transactions.aggregate(pipeline))
    return result[0] if result else None

In [92]:
# Example use 
listing_id = sample_tx_listing
start_date = datetime(2008, 1, 1)
end_date = datetime(2010, 12, 31)

result = get_average_transaction_price(listing_id, start_date, end_date)
pprint(result)

{'_id': ObjectId('68054a2843376708823c1712'),
 'average_price': 53.51,
 'transaction_count': 3}


11)	We wish to have a summary webpage that displays information about our top destinations. This webpage should display for each of the top 10 cities some basic information about our operations in the area (number of properties by type for example, average price by type) but you can choose the metrics. For each of the top 10 cities it should also provide some basic information about the top 3 properties in each city (price, number of review, whatever you think useful) to show an example of the properties available in the area. We would like to keep this webpage up to date as information changes.

First, we will get the list of top 10 cities based on the number of properties (listings). We want to display city name, its country, total number of listings, as well as the average price and number of listings by property type

In [93]:
# grouping by city and property type to get average price and count by property type
# adding country as "_id" in case in the future there will be cities with the same name located in different countries
query_1 = {"$group": {
        "_id": {
            "city": "$City",
            "country": "$Address.country",
            "property_type": "$PropertyType"
        },
        "average_price": {"$avg": "$Price"},
        "properties_count": {"$count": {}}
    }
}

# grouping by city (and country) to get the total number of listings and locate data by property type in an array
query_2 = {"$group": {
        "_id": {
            "city": "$_id.city",
            "country": "$_id.country"
        },
        "number_of_listings": {"$sum": "$properties_count"},
        "property_types": {
            "$push": {
                "PropertyType": "$_id.property_type",
                "average_price": {"$round": ["$average_price", 2]}, #round to the second floating poing to avoid weird numbers
                "number_properties": "$num_properties"
            }
        }
    }
}

# using projection to organize the fields in the output in desired structure
query_3 = {"$project": {
        "_id": 0,
        "city": "$_id.city",
        "country": "$_id.country",
        "number_of_listings": 1,
        "property_types": 1
    }
}

# sorting in descending order by number of listings
query_4 = {"$sort": {"number_of_listings": -1}}

#getting the top 10
query_5 = {"$limit": 10}

pipeline_top_cities = [query_1, query_2, query_3, query_4, query_5]

result = db.listings.aggregate(pipeline_top_cities)

top_cities = []
for r in result:
    top_cities.append(r['city'])
    pprint(r)



{'city': 'Istanbul',
 'country': 'Turkey',
 'number_of_listings': 661,
 'property_types': [{'PropertyType': 'Aparthotel', 'average_price': 251.27},
                    {'PropertyType': 'Guest suite', 'average_price': 121.0},
                    {'PropertyType': 'Serviced apartment',
                     'average_price': 314.42},
                    {'PropertyType': 'Boutique hotel', 'average_price': 290.67},
                    {'PropertyType': 'Loft', 'average_price': 229.12},
                    {'PropertyType': 'House', 'average_price': 276.3},
                    {'PropertyType': 'Hotel', 'average_price': 923.5},
                    {'PropertyType': 'Villa', 'average_price': 1383.17},
                    {'PropertyType': 'Bed and breakfast',
                     'average_price': 594.77},
                    {'PropertyType': 'Casa particular (Cuba)',
                     'average_price': 170.8},
                    {'PropertyType': 'Apartment', 'average_price': 365.98},
            

In [94]:
print(top_cities)

['Istanbul', 'Montreal', 'Barcelona', 'Sydney', 'New York', 'Rio De Janeiro', 'Hong Kong', 'Porto', 'Oahu', 'Maui']


Next, we will extract information about top 3 properties in each city. We will use an aggregations pipeline to get 3 properties with the highest rating (as in task 9 we assumed it to be a more fine-grained quality metric), for which we implemented ReviewScore.Rating index, and some information about them, such as number of bedrooms, bathrooms and visitors it can accommodate, number of reviews and rating. We will wrap the pipeline into a function which will take a city name as input and will call the function for each city from the top cities list.

In [95]:
def get_top_properties(city, k=3):
    ''' 
    Applies an aggregation pipeline to the listings collection to provide summary about top k properties (based on rating) of a given city
    :param city (str) - name of the city
    :param k(int, default =3) - number of top properties
    '''
    query_1 = {"$match": {"City": city}}

    query_2 = {"$sort": {"ReviewScore.Rating": -1}}

    query_3 = {"$limit": k}

    # joining with the reviews collection to be able to count reviews
    query_4 = {"$lookup":{
           "from": "reviews",
           "localField": "_id", 
           "foreignField": "ListingId", 
           "as": "reviews" 
        } }

    #organizing fields needed in the input
    query_5 = {"$project": {
                "Name": 1,
                "Price": 1,
                "Bedrooms": 1,
                "Bathrooms": 1,
                "Accommodates": 1,   
                "number_reviews": { "$size": "$reviews" },
                "Rating": "$ReviewScore.Rating"
            }
    }

    pipeline = [query_1, query_2, query_3, query_4, query_5]

    result = db.listings.aggregate(pipeline)

    print(city)
    i = 1
    for r in result:
        summary = f"{i}. {r['Name']} for up to {r['Accommodates']} guests with {r['Bedrooms']} bedroom(s) and {r['Bathrooms']} bathroom(s) for {r['Price']} eur per night. Rating of {r['Rating']} based on {r['number_reviews']} reviews. "
        print(summary)
        i+=1

In [96]:
for city in top_cities:
    get_top_properties(city)

Istanbul
1. Byzantium Suites Hotel for up to 2 guests with 1 bedroom(s) and 1.0 bathroom(s) for 269.0 eur per night. Rating of 100 based on 3 reviews. 
2. Room in Kurtuluş close to subway for up to 2 guests with 1 bedroom(s) and 1.0 bathroom(s) for 79.0 eur per night. Rating of 100 based on 1 reviews. 
3. Cosy Room in Kadıkoy for up to 2 guests with 1 bedroom(s) and 1.0 bathroom(s) for 63.0 eur per night. Rating of 100 based on 1 reviews. 
Montreal
1. SOSHE 210, Adj. MUHC Glen Campus for up to 2 guests with 0 bedroom(s) and 1.0 bathroom(s) for 50.0 eur per night. Rating of 100 based on 4 reviews. 
2. Private & luminous room for up to 2 guests with 1 bedroom(s) and 1.0 bathroom(s) for 30.0 eur per night. Rating of 100 based on 13 reviews. 
3. Luxurious Appartement in Downtown Montreal for up to 5 guests with 1 bedroom(s) and 1.0 bathroom(s) for 400.0 eur per night. Rating of 100 based on 3 reviews. 
Barcelona
1. Comfortable and sunny double room for up to 2 guests with 1 bedroom(s) and 

12) Add a new property with a new host in one of the top 10 cities. The host selects the top 10 most common amenities to list.

For a new property creation, as the target city we opt for one of the cities from the previous task, New York. We will check the top 10 amenities.

In [97]:
# Top ten amenities 
pipeline = [
    { "$unwind": "$Amenities" },
    { "$group": { "_id": "$Amenities", "count": { "$sum": 1 } } },
    { "$sort": { "count": -1 } },
    { "$limit": 10 }
]

top_amenities = list(db.listings.aggregate(pipeline))

print("Top 10 amenities by usage:")
for a in top_amenities:
    print(f"- {a['_id']} ({a['count']} listings)")


Top 10 amenities by usage:
- Wifi (5303 listings)
- Essentials (5048 listings)
- Kitchen (4951 listings)
- TV (4280 listings)
- Hangers (4226 listings)
- Hair dryer (3900 listings)
- Washer (3877 listings)
- Shampoo (3709 listings)
- Iron (3692 listings)
- Laptop friendly workspace (3442 listings)


In [98]:
# the list of top_amenities
top_amenities = [a["_id"] for a in db.listings.aggregate(pipeline)] 
top_amenities

['Wifi',
 'Essentials',
 'Kitchen',
 'TV',
 'Hangers',
 'Hair dryer',
 'Washer',
 'Shampoo',
 'Iron',
 'Laptop friendly workspace']

First, we need to create a new host located in New York. Such values as IsSUperhost, ResponseRate/ResponseTime and TotalListingsCount will be initialized as False, None (analogue of null in python) and 0, since they require some previous record of activities on the webpage. For simplicity, let's assume that identity and other verifications happen automatically immediately at registration, equivalent to the moment of creation of a new document in the hosts collection. 

In [99]:
new_host = db.hosts.insert_one({

    "Name": "Poligraf",
    "Location": "New York, New York, United States",
    "IsSuperhost": False,
    "ResponseRate": None,
    "ResponseTime": None,
    "TotalListingsCount": 0,
    "HasProfilePic": True,
    "IdentityVerified": True,
    "About": "I love hosting travelers and help them explore the city.",
    "Url": "https://www.airbnb.com/users/show/987654321",
    "Verifications": [
        "email",
        "phone",
        "government_id"
    ],
    "Neighbourhood": "Queens",
    "HostCountry" : "United States",
})

host = db.hosts.find_one({"Name": "Poligraf"})
new_host_id = host.get('_id')
pprint(host)

{'About': 'I love hosting travelers and help them explore the city.',
 'HasProfilePic': True,
 'HostCountry': 'United States',
 'IdentityVerified': True,
 'IsSuperhost': False,
 'Location': 'New York, New York, United States',
 'Name': 'Poligraf',
 'Neighbourhood': 'Queens',
 'ResponseRate': None,
 'ResponseTime': None,
 'TotalListingsCount': 0,
 'Url': 'https://www.airbnb.com/users/show/987654321',
 'Verifications': ['email', 'phone', 'government_id'],
 '_id': ObjectId('68054cb44337670882441cf9')}


Now we can add a new property from the newly added host creating a new document in the listings collection with the link referencing host "_id"

In [100]:
new_property= db.listings.insert_one({
    "Name": "Modern apartment in Queens",
    "Address": {
        "street": "Queens, NY, United States",
        "suburb": "Queens",
        "government_area": "Long Island City",
        "market": "New York",
        "country" : "United States",
        "country_code" : "US",
        "location" : {
            "type" : "Point",
            "coordinates" : [
                -75.12345,
                42.54321
            ],
            "is_location_exact" : False
            }
        },
    "City": "New York",  
    "Images": "https://a0.muscache.com/im/pictures/b1c096e3-25d5-404a-a409-f8db12c690b5.jpg?aki_policy=large",
    "Price": 125.0,
    "RoomType": "Entire home/apt",
    "PropertyType": "Apartment",
    "Beds": 1,
    "Bedrooms": 1,
    "Bathrooms": 1.0,
    "Accommodates": 2,
    "Amenities": top_amenities,
    "Summary": "Cozy little flat with plenty of daylight",
    "ReviewScore": { 
        "Cleanliness" : None,
        "Communication" : None,
        "Checkin" : None,
        "Location" : None,
        "Value" : None
        },
    "HostId": new_host_id,
    "BedType": "Real Bed",
    "SecurityDeposit": 250.0,
    "MaximumNights": 30,
    "MinimumNights": 5,
    "ListingURL": "https://www.airbnb.com/rooms/987654321",
    "SampleReviews": []
})



In [101]:
property = db.listings.find_one({"HostId": new_host_id}) 
new_listing_id = property.get('_id')
pprint(property)

{'Accommodates': 2,
 'Address': {'country': 'United States',
             'country_code': 'US',
             'government_area': 'Long Island City',
             'location': {'coordinates': [-75.12345, 42.54321],
                          'is_location_exact': False,
                          'type': 'Point'},
             'market': 'New York',
             'street': 'Queens, NY, United States',
             'suburb': 'Queens'},
 'Amenities': ['Wifi',
               'Essentials',
               'Kitchen',
               'TV',
               'Hangers',
               'Hair dryer',
               'Washer',
               'Shampoo',
               'Iron',
               'Laptop friendly workspace'],
 'Bathrooms': 1.0,
 'BedType': 'Real Bed',
 'Bedrooms': 1,
 'Beds': 1,
 'City': 'New York',
 'HostId': ObjectId('68054cb44337670882441cf9'),
 'Images': 'https://a0.muscache.com/im/pictures/b1c096e3-25d5-404a-a409-f8db12c690b5.jpg?aki_policy=large',
 'ListingURL': 'https://www.airbnb.com/rooms/98

With the addition of new listing, the field "TotalListingsCount" of the corresponding host in hosts collection (computed pattern, since new properties are registered not so frequently as properties are searched and booked, staleness and recomputation are solved by incrementation on the application side instead of periodically recalculating the count) must be incremented by one:

In [102]:
result = db.hosts.update_one({"_id": new_host_id}, {"$inc": {"TotalListingsCount": 1}})
pprint(db.hosts.find_one({"_id": new_host_id}))

{'About': 'I love hosting travelers and help them explore the city.',
 'HasProfilePic': True,
 'HostCountry': 'United States',
 'IdentityVerified': True,
 'IsSuperhost': False,
 'Location': 'New York, New York, United States',
 'Name': 'Poligraf',
 'Neighbourhood': 'Queens',
 'ResponseRate': None,
 'ResponseTime': None,
 'TotalListingsCount': 1,
 'Url': 'https://www.airbnb.com/users/show/987654321',
 'Verifications': ['email', 'phone', 'government_id'],
 '_id': ObjectId('68054cb44337670882441cf9')}


We can see that the TotalListingCount of the new host is now equal to 1.

13) Add a new review from one of our top 20 reviewers for this new property.

First, we randomly select one of the top 20 reviewers

In [103]:
random.seed(42)

random_reviewer = random.choice(top_reviewers)
print(random_reviewer)

{'_id': ObjectId('68054bcd43376708824066ad'), 'Name': 'Thien', 'ReviewCount': 9}


Next, we insert a new document to the reviews collection with a link referencing the corresponding new property. 

In [104]:
new_review = db.reviews.insert_one({
    "ListingId" : new_listing_id,
    "ReviewerId" : random_reviewer["_id"],
    "ReviewerName" : random_reviewer["Name"],
    "Date" : datetime.now(),
    "Comments" : "Overall, good value for money. The kitchen is well-equipped, but Wifi stability could be improved." 
})



In [105]:
review = db.reviews.find_one({"ListingId": new_listing_id})
new_review_id = review.get('_id')
pprint(new_review_id)

ObjectId('68054cb54337670882441cfb')


We want the review to appear among the sample reviews in the property document in the listings collection and assume the reviewer also left review scores:

In [ ]:
review_doc = db.reviews.find_one({"_id": new_review_id}, {
    "ReviewerName": 1,
    "Date": 1,
    "Comments": 1,
    "_id": 0 
})

add_review = db.listings.update_one({"_id": new_listing_id}, {"$push": {"SampleReviews": review_doc}})

# since it is the first review, metrics are added directly as is. with more reviews being added, each score will
# need to be averaged by the number of reviews
add_rating = db.listings.update_one({"_id": new_listing_id}, {"$set": {
    "ReviewScore.Checkin": 5,
    "ReviewScore.Cleanliness": 10, 
    "ReviewScore.Communication": 8, 
    "ReviewScore.Location": 4, 
    "ReviewScore.Rating": 85, 
    "ReviewScore.Value": 10
    }
}
)

property = db.listings.find({"_id": new_listing_id}) 

for x in property: 
    pprint(x)

{'Accommodates': 2,
 'Address': {'country': 'United States',
             'country_code': 'US',
             'government_area': 'Long Island City',
             'location': {'coordinates': [-75.12345, 42.54321],
                          'is_location_exact': False,
                          'type': 'Point'},
             'market': 'New York',
             'street': 'Queens, NY, United States',
             'suburb': 'Queens'},
 'Amenities': ['Wifi',
               'Essentials',
               'Kitchen',
               'TV',
               'Hangers',
               'Hair dryer',
               'Washer',
               'Shampoo',
               'Iron',
               'Laptop friendly workspace'],
 'Bathrooms': 1.0,
 'BedType': 'Real Bed',
 'Bedrooms': 1,
 'Beds': 1,
 'City': 'New York',
 'HostId': ObjectId('68054cb44337670882441cf9'),
 'Images': 'https://a0.muscache.com/im/pictures/b1c096e3-25d5-404a-a409-f8db12c690b5.jpg?aki_policy=large',
 'ListingURL': 'https://www.airbnb.com/rooms/98

If we rerun the query from the task 8 now, we will see that the number of reviews by the reviewer Thien has now incremented by 1 and is equal to 10 (used to be 9). Reviews are expected to be accessed for both writes and reads much more frequently than host or listing writes are expected to happen, top reviewers webpage should be kept up to date, and reviews collection has index on ReviewerId, we prefer not to precompute or approximate number of reviews by reviewers but update dynamically, leveraging the index. 

In [110]:
# from task 8. Note number of reviews of reviewer with name Thien in the output
pipeline = [
    {
        "$group": {
            "_id": "$ReviewerId",
            "Name": { "$first": "$ReviewerName" },
            "ReviewCount": { "$sum": 1 }
        }
    },
    { "$sort": { "ReviewCount": -1 } },
    { "$limit": 20 }
]

top_reviewers = list(db.reviews.aggregate(pipeline))

for reviewer in top_reviewers:
    print(f"{reviewer['Name']} (ID: {reviewer['_id']}) – {reviewer['ReviewCount']} reviews")

Filipe (ID: 68054bcb4337670882405cd0) – 24 reviews
Nick (ID: 68054c1e433767088241f06f) – 13 reviews
Uge (ID: 68054c034337670882416545) – 10 reviews
Thien (ID: 68054bcd43376708824066ad) – 10 reviews
Lisa (ID: 68054c3d4337670882428a06) – 9 reviews
Courtney (ID: 68054bd443376708824083d6) – 8 reviews
David (ID: 68054bb443376708823fe774) – 8 reviews
Todd (ID: 68054bf64337670882411e76) – 8 reviews
Jodi (ID: 68054c634337670882433a27) – 8 reviews
Lisa (ID: 68054c1e433767088241f067) – 8 reviews
Lance (ID: 68054bb143376708823fdcb6) – 7 reviews
David (ID: 68054c6543376708824341f5) – 7 reviews
Mary (ID: 68054bff4337670882414cb3) – 6 reviews
Karen (ID: 68054bc64337670882404360) – 6 reviews
Assis (ID: 68054c604337670882433516) – 6 reviews
Branden (ID: 68054c2443376708824212ca) – 6 reviews
Dan (ID: 68054bef433767088240f4b5) – 6 reviews
Pierre (ID: 68054bc0433767088240225a) – 6 reviews
Erik (ID: 68054bff4337670882415072) – 6 reviews
Chris (ID: 68054c7b433767088243aebd) – 6 reviews


14) Add a new review metric called 'x_factor' with a score of 10. Show that the average score across all metrics is correctly calculated for this listing, using the previously developed query.

In [107]:
# checking average score of the newly created and reviewed property
result = get_average_review_score(new_listing_id)
pprint(result)

{'ReviewScore': {'Checkin': 5,
                 'Cleanliness': 10,
                 'Communication': 8,
                 'Location': 4,
                 'Rating': 85,
                 'Value': 10},
 '_id': ObjectId('68054cb44337670882441cfa'),
 'average_score': 7.4}


In [108]:
# adding new metric x_factor
db.listings.update_one(
    {"_id": new_listing_id },
    { "$set": { "ReviewScore.x_factor": 10 } }
)

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [109]:
# Check if it changed
result = get_average_review_score(new_listing_id)
pprint(result)

{'ReviewScore': {'Checkin': 5,
                 'Cleanliness': 10,
                 'Communication': 8,
                 'Location': 4,
                 'Rating': 85,
                 'Value': 10,
                 'x_factor': 10},
 '_id': ObjectId('68054cb44337670882441cfa'),
 'average_score': 7.83}


We can see that the average score changed after the new metric was added.